In [1]:
import pandas as pd
import geopandas as gpd
from osgeo import gdal
import numpy as np
import matplotlib.pyplot as plt
from shapely import geometry
from shapely.geometry import Point
from shapely.ops import transform
import geopandas
import shapely as shp
import numpy as np
import pyproj
from sklearn.neighbors import BallTree
import warnings
warnings.filterwarnings("ignore")
from varname import nameof
from skimage.filters import threshold_otsu
import json
import statistics

In [2]:
data_individual = pd.read_excel("03._Data_combined.xlsx")
df_di = pd.DataFrame(data_individual)
df_di.head(1)

,Extract ID,Paragraph ID,Interview ID,Text,Start Offset,End Offset,Sentiment,Is Interesting,Annotations,Paragraph Number,...,trust,travel_to_major_city,travel_outside_uganda,travel_outside_uganda_specifications,no_days_away_from_home_last_year,reason_for_absence,radio_listening_frequency,tv_watching_frequency,newspaper_reading_frequency,internet_use_frequency
0,60645a53a2237b3f54690d27,60645a53a2237b3f54690d26,60645a53a2237b3f54690d25,Use it to collect rubbish and weeds dug out in...,0.0,55.0,Postive,True,Being healthy,0.0,...,1.0,2.0,1.0,NaN,3,0,1.0,5.0,4.0,5.0


In [3]:
#Get specific columns
df_di_selected = df_di[['Interview ID', 'Village', 'lat,long', 'age', 'sex', 'Text', 'Item Name', 'Annotations', 
                        'no_people_in_household', 'highest_grade_completed','distance_fetch_water_one_way',
                        'cooking_energy_source', 'water_source', 'toilet_type', ]].copy()
df_di_selected.head()
#Absent parameters
#'electrical_connection'
#'financial_satisfaction'
#'radio_listening_frequency', 
#'tv_watching_frequency',                       
#'newspaper_reading_frequency', 
#'internet_use_frequency'

,Interview ID,Village,"lat,long",age,sex,Text,Item Name,Annotations,no_people_in_household,highest_grade_completed,distance_fetch_water_one_way,cooking_energy_source,water_source,toilet_type
0,60645a53a2237b3f54690d25,Lama,"3.668252174,31.92603986",24.0,2,Use it to collect rubbish and weeds dug out in...,Hoe,Being healthy,3,1,2,1,6,2
1,60645a53a2237b3f54690d25,Lama,"3.668252174,31.92603987",24.0,2,It makes it easy for rubbish that is heavy to ...,Hoe,Unburden,3,1,2,1,6,2
2,60645a53a2237b3f54690d25,Lama,"3.668252174,31.92603988",24.0,2,income and you can sew it for a dress.,Hoe,Income,3,1,2,1,6,2
3,60645a53a2237b3f54690d25,Lama,"3.668252174,31.92603989",24.0,2,This is what would cost me more than one journ...,Motorcycle,Fixed costs,3,1,2,1,6,2
4,60645a53a2237b3f54690d25,Lama,"3.668252174,31.92603990",24.0,2,These days if you have motorcycle you can use ...,Motorcycle,Income,3,1,2,1,6,2


In [4]:
df_values = df_di_selected['Annotations'].str.split(';', expand=True)
df_values2 = df_values.rename(columns={0:"Val_1", 1: "Val_2", 2: "Val_3", 3: "Val_4", 4: "Val_5", 5: "Val_6", 
                                       6: "Val_7",7: "Val_8", 8: "Val_9", 9: "Val_10", 10: "Val_11", 11: "Val_12",
                                       12: "Val_13", 13: "Val_14"})
df_values2

,Val_1,Val_2,Val_3,Val_4,Val_5,Val_6,Val_7,Val_8,Val_9,Val_10,Val_11,Val_12,Val_13,Val_14
0,Being healthy,None,None,None,None,None,None,None,None,None,None,None,None,None
1,Unburden,None,None,None,None,None,None,None,None,None,None,None,None,None
2,Income,None,None,None,None,None,None,None,None,None,None,None,None,None
3,Fixed costs,None,None,None,None,None,None,None,None,None,None,None,None,None
4,Income,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6220,Unburden,Comfort,Togetherness (loved ones),None,None,None,None,None,None,None,None,None,None,None
6221,Continuous costs,None,None,None,None,None,None,None,None,None,None,None,None,None
6222,Aesthetics,None,None,None,None,None,None,None,None,None,None,None,None,None
6223,Ease of use,None,None,None,None,None,None,None,None,None,None,None,None,None


In [5]:
#Obtention of the final dataframe 
df_di_selected2 = pd.concat([df_di_selected, df_values2], axis=1)
del df_di_selected2['Annotations']
df_di_selected2.to_excel("00._output_df01_S_PhD_Individual_chosen_values_v0.xlsx")
df_di_selected2.head(1)

,Interview ID,Village,"lat,long",age,sex,Text,Item Name,no_people_in_household,highest_grade_completed,distance_fetch_water_one_way,...,Val_5,Val_6,Val_7,Val_8,Val_9,Val_10,Val_11,Val_12,Val_13,Val_14
0,60645a53a2237b3f54690d25,Lama,"3.668252174,31.92603986",24.0,2,Use it to collect rubbish and weeds dug out in...,Hoe,3,1,2,...,None,None,None,None,None,None,None,None,None,None


In [6]:
#Delete all rows that have not 'stove', 'gas stove', 'Hospital' or 'School', and counting number of obsevations
df_di_selected3 = df_di_selected2.loc[df_di_selected2['Item Name'].isin(['Stove','Gas Stove'])].reset_index()
unique_values_ID = pd.unique(df_di_selected3['Interview ID'])
print("There are", len(unique_values_ID), "observations, and the DF has", len(df_di_selected3.index), "rows")
df_di_selected3.to_excel("00._output_df02_S_PhD_Individual_Stove_Gas_Stove.xlsx")#This file is copied in folder and modified later with parameters
df_di_selected3.head(1)
#unique_values_ID

There are 26 observations, and the DF has 62 rows


,index,Interview ID,Village,"lat,long",age,sex,Text,Item Name,no_people_in_household,highest_grade_completed,...,Val_5,Val_6,Val_7,Val_8,Val_9,Val_10,Val_11,Val_12,Val_13,Val_14
0,87,60645a4ea2237b3f54690cef,Bwindi,"-0.9682629102,29.61722332",32.0,1,It can help me at home to cook faster and can ...,Gas Stove,7,2,...,None,None,None,None,None,None,None,None,None,None


In [7]:
#pd_new_data.index.values[2]
#pd_new_data.iloc[4,0]

#Construction of the Observations Data Frame

#Interviews filtering

pd_new_data = pd.DataFrame()
for column in df_di_selected3: 
    if column == 'Interview ID':
        i=0
        print('Yes, I found the', column, 'column')
        j = df_di_selected3.columns.get_loc(column)
        print('The column position of', column, 'is', j)
        pd_new_data.at[i, column] = df_di_selected3.loc[i,'Interview ID']
        i=i+1
        for row in df_di_selected3.iterrows():
            if df_di_selected3.loc[i,'Interview ID']!=df_di_selected3.loc[i-1,'Interview ID']:
                pd_new_data.at[i, column] = df_di_selected3.loc[i,'Interview ID'] 
            i=i+1
            if i == len(df_di_selected3.index)-1:
                break

unique_values_ID = pd.unique(pd_new_data['Interview ID'])
print("There are", len(unique_values_ID), "observations")

#Matching the Village with the interviewer

i=0
for row in pd_new_data.iterrows():
    pd_new_data.at[pd_new_data.index.values[i], 'Village'] = df_di_selected3.loc[pd_new_data.index.values[i],'Village'] 
    i=i+1
    if i == len(df_di_selected3.index)-1:
        break

#Matching the Item Name with the interviewer

i=0
for row in pd_new_data.iterrows():
    pd_new_data.at[pd_new_data.index.values[i], 'Item Name'] = df_di_selected3.loc[pd_new_data.index.values[i],'Item Name'] 
    i=i+1
    if i == len(df_di_selected3.index)-1:
        break  

#Matching the latitude and longitude with the interviewer

i=0
for row in pd_new_data.iterrows():
    pd_new_data.at[pd_new_data.index.values[i], 'lat,long'] = df_di_selected3.loc[pd_new_data.index.values[i],'lat,long'] 
    i=i+1
    if i == len(df_di_selected3.index)-1:
        break   

#Matching the age with the interviewer

i=0
for row in pd_new_data.iterrows():
    pd_new_data.at[pd_new_data.index.values[i], 'age'] = df_di_selected3.loc[pd_new_data.index.values[i],'age'] 
    i=i+1
    if i == len(df_di_selected3.index)-1:
        break         
        
#Matching the sex with the interviewer

i=0
for row in pd_new_data.iterrows():
    pd_new_data.at[pd_new_data.index.values[i], 'sex'] = df_di_selected3.loc[pd_new_data.index.values[i],'sex'] 
    i=i+1
    if i == len(df_di_selected3.index)-1:
        break 
        
#Matching the number of people in household with the interviewer

i=0
for row in pd_new_data.iterrows():
    pd_new_data.at[pd_new_data.index.values[i], 'no_people_in_household'] = df_di_selected3.loc[pd_new_data.index.values[i],'no_people_in_household'] 
    i=i+1
    if i == len(df_di_selected3.index)-1:
        break        

#Matching the highest grade completed with the interviewer

i=0
for row in pd_new_data.iterrows():
    pd_new_data.at[pd_new_data.index.values[i], 'highest_grade_completed'] = df_di_selected3.loc[pd_new_data.index.values[i],'highest_grade_completed'] 
    i=i+1
    if i == len(df_di_selected3.index)-1:
        break       

#Matching the distance to water source with the interviewer

i=0
for row in pd_new_data.iterrows():
    pd_new_data.at[pd_new_data.index.values[i], 'distance_fetch_water_one_way'] = df_di_selected3.loc[pd_new_data.index.values[i],'distance_fetch_water_one_way'] 
    i=i+1
    if i == len(df_di_selected3.index)-1:
        break 

#Matching the connection to electricity with the interviewer

#i=0
#for row in pd_new_data.iterrows():
#    pd_new_data.at[pd_new_data.index.values[i], 'electrical_connection'] = df_di_selected3.loc[pd_new_data.index.values[i],'electrical_connection'] 
#    i=i+1
#    if i == len(df_di_selected3.index)-1:
#        break    
        
#Matching the financial satisfaction with the interviewer

#i=0
#for row in pd_new_data.iterrows():
#    pd_new_data.at[pd_new_data.index.values[i], 'financial_satisfaction'] = df_di_selected3.loc[pd_new_data.index.values[i],'financial_satisfaction'] 
#    i=i+1
#    if i == len(df_di_selected3.index)-1:
#        break  
        
#Matching the type of fuel with the interviewer

i=0
for row in pd_new_data.iterrows():
    if df_di_selected3.loc[pd_new_data.index.values[i],'cooking_energy_source'] == '1;2':
        pd_new_data.at[pd_new_data.index.values[i], 'type_of_fuel'] = 6
    elif df_di_selected3.loc[pd_new_data.index.values[i],'cooking_energy_source'] == '1;3':
        pd_new_data.at[pd_new_data.index.values[i], 'type_of_fuel'] = 7
    else:
        pd_new_data.at[pd_new_data.index.values[i], 'type_of_fuel'] = df_di_selected3.loc[pd_new_data.index.values[i],'cooking_energy_source'] 
    i=i+1
    if i == len(df_di_selected3.index)-1:
        break

#Matching the source of water with the interviewer

i=0
for row in pd_new_data.iterrows():
    if df_di_selected3.loc[pd_new_data.index.values[i],'water_source'] == '6;11':
        pd_new_data.at[pd_new_data.index.values[i], 'water_source'] = 13
    elif df_di_selected3.loc[pd_new_data.index.values[i],'water_source'] == '9;11':
        pd_new_data.at[pd_new_data.index.values[i], 'water_source'] = 14
    elif df_di_selected3.loc[pd_new_data.index.values[i],'water_source'] == '1;9':
        pd_new_data.at[pd_new_data.index.values[i], 'water_source'] = 15
    elif df_di_selected3.loc[pd_new_data.index.values[i],'water_source'] == '1;7':
        pd_new_data.at[pd_new_data.index.values[i], 'water_source'] = 16
    elif df_di_selected3.loc[pd_new_data.index.values[i],'water_source'] == '4;12':
        pd_new_data.at[pd_new_data.index.values[i], 'water_source'] = 17
    elif df_di_selected3.loc[pd_new_data.index.values[i],'water_source'] == '3;4':
        pd_new_data.at[pd_new_data.index.values[i], 'water_source'] = 18
    elif df_di_selected3.loc[pd_new_data.index.values[i],'water_source'] == '7;9':
        pd_new_data.at[pd_new_data.index.values[i], 'water_source'] = 19
    elif df_di_selected3.loc[pd_new_data.index.values[i],'water_source'] == '6;9;11':
        pd_new_data.at[pd_new_data.index.values[i], 'water_source'] = 20
    elif df_di_selected3.loc[pd_new_data.index.values[i],'water_source'] == '4;9':
        pd_new_data.at[pd_new_data.index.values[i], 'water_source'] = 21
    else:
        pd_new_data.at[pd_new_data.index.values[i], 'water_source'] = df_di_selected3.loc[pd_new_data.index.values[i],'water_source'] 
    i=i+1
    if i == len(df_di_selected3.index)-1:
        break  
        
#Matching the type of toilet with the interviewer

i=0
for row in pd_new_data.iterrows():
    if df_di_selected3.loc[pd_new_data.index.values[i],'toilet_type'] == '1;2':
        pd_new_data.at[pd_new_data.index.values[i], 'toilet_type'] = 6
    else:    
        pd_new_data.at[pd_new_data.index.values[i], 'toilet_type'] = df_di_selected3.loc[pd_new_data.index.values[i],'toilet_type'] 
    i=i+1
    if i == len(df_di_selected3.index)-1:
        break
        
#Matching the frecuency of listening to the radio with the interviewer

#i=0
#for row in pd_new_data.iterrows():
#    pd_new_data.at[pd_new_data.index.values[i], 'radio_listening_frequency'] = df_di_selected3.loc[pd_new_data.index.values[i],'radio_listening_frequency'] 
#    i=i+1
#    if i == len(df_di_selected3.index)-1:
#        break  
        
#Matching the frecuency of watching tv with the interviewer

#i=0
#for row in pd_new_data.iterrows():
#    pd_new_data.at[pd_new_data.index.values[i], 'tv_watching_frequency'] = df_di_selected3.loc[pd_new_data.index.values[i],'tv_watching_frequency'] 
#    i=i+1
#    if i == len(df_di_selected3.index)-1:
#        break         
        
#Matching the frecuency of watching tv with the interviewer

#i=0
#for row in pd_new_data.iterrows():
#    pd_new_data.at[pd_new_data.index.values[i], 'newspaper_reading_frequency'] = df_di_selected3.loc[pd_new_data.index.values[i],'newspaper_reading_frequency'] 
#    i=i+1
#    if i == len(df_di_selected3.index)-1:
#        break 
        
#Matching the frecuency of watching tv with the interviewer

#i=0
#for row in pd_new_data.iterrows():
#    pd_new_data.at[pd_new_data.index.values[i], 'internet_use_frequency'] = df_di_selected3.loc[pd_new_data.index.values[i],'internet_use_frequency'] 
#    i=i+1
#    if i == len(df_di_selected3.index)-1:
#        break 
        
#Creating columns for the chosen items to see if that person has chosen a specific item
    
#pd_new_data.insert(1,"Stove", "x")
#pd_new_data.insert(2,"Dishes", "x")
#pd_new_data.insert(3,"Saucepan", "x")
#pd_new_data.insert(3,"Toilet- flush", "x")
#pd_new_data.insert(4,"Cow", "x")
#pd_new_data.insert(5,"Grain Mill", "x")
#pd_new_data.insert(6,"Rainwater Harvesting System", "x")
#pd_new_data.insert(7,"Water Point", "x")
#pd_new_data.insert(8,"Borehole", "x")
#pd_new_data.insert(9,"Kerosene Lantern", "x")
#pd_new_data.insert(10,"Solar Lantern", "x")
#pd_new_data.insert(11,"Pig", "x")
#pd_new_data.insert(12,"Sugar Cane", "x")
#pd_new_data.insert(13,"Water Pot", "x")
#pd_new_data.insert(14,"Crops (Maize-rice)", "x")
#pd_new_data.insert(15,"Candle", "x")
#pd_new_data.insert(16,"Generator", "x")

#Saving Data frae for only geo data

#df_only_geo = pd_new_data[['Interview ID', 'Village', 'lat,long']].copy()

#Matching the selected items by the interviewer

#i=0
#ii=0
#while i < len(df_di_selected3)-1:
#    if df_di_selected3.at[df_di_selected3.index.values[i], 'Interview ID'] == df_di_selected3.at[df_di_selected3.index.values[i+1], 'Interview ID']: 
#        item = df_di_selected3.at[df_di_selected3.index.values[i], 'Item Name']
#        if (item == 'Stove') | (item == 'Gas Stove'):
#            pd_new_data.at[pd_new_data.index[ii],'Stove'] = 1
#        if item == 'Dishes':
#            pd_new_data.at[pd_new_data.index[ii],'Dishes'] = 1        
#        if item == 'Saucepan':
#            pd_new_data.at[pd_new_data.index[ii],'Saucepan'] = 1                 
        #if item == 'Solar PV System':
        #    pd_new_data.at[pd_new_data.index[ii],'Solar PV System'] = 1
        #if item == 'Toilet- flush':
        #    pd_new_data.at[pd_new_data.index[ii],'Toilet- flush'] = 1
        #if item == 'Cow':
        #    pd_new_data.at[pd_new_data.index[ii],'Cow'] = 1
        #if item == 'Grain Mill':
        #    pd_new_data.at[pd_new_data.index[ii],'Grain Mill'] = 1
        #if item == 'Rainwater Harvesting System':
        #    pd_new_data.at[pd_new_data.index[ii],'Rainwater Harvesting System'] = 1
        #if item == 'Water Point':
        #    pd_new_data.at[pd_new_data.index[ii],'Water Point'] = 1
        #if item == 'Borehole':
        #    pd_new_data.at[pd_new_data.index[ii],'Borehole'] = 1
        #if item == 'Kerosene Lantern':
        #    pd_new_data.at[pd_new_data.index[ii],'Kerosene Lantern'] = 1
        #if item == 'Solar Lantern':
        #    pd_new_data.at[pd_new_data.index[ii],'Solar Lantern'] = 1
        #if item == 'Pig':
        #   pd_new_data.at[pd_new_data.index[ii],'Pig'] = 1
        #if item == 'Sugar Cane':
        #    pd_new_data.at[pd_new_data.index[ii],'Sugar Cane'] = 1
        #if item == 'Water Pot':
        #    pd_new_data.at[pd_new_data.index[ii],'Water Pot'] = 1
        #if item == 'Crops (Maize-rice)':
        #    pd_new_data.at[pd_new_data.index[ii],'Crops (Maize-rice)'] = 1
        #if item == 'Candle':
        #    pd_new_data.at[pd_new_data.index[ii],'Candle'] = 1
        #if item == 'Generator':
        #    pd_new_data.at[pd_new_data.index[ii],'Generator'] = 1
            
#    else:
#        item = df_di_selected3.at[df_di_selected3.index.values[i], 'Item Name']
#        if (item == 'Stove') | (item == 'Gas Stove'):
#            pd_new_data.at[pd_new_data.index[ii],'Stove'] = 1
#        if item == 'Dishes':
#            pd_new_data.at[pd_new_data.index[ii],'Dishes'] = 1
#        if item == 'Saucepan':
#            pd_new_data.at[pd_new_data.index[ii],'Saucepan'] = 1
        #if item == 'Solar PV System':
        #    pd_new_data.at[pd_new_data.index[ii],'Solar PV System'] = 1
        #if item == 'Toilet- flush':
        #    pd_new_data.at[pd_new_data.index[ii],'Toilet- flush'] = 1
        #if item == 'Cow':
        #    pd_new_data.at[pd_new_data.index[ii],'Cow'] = 1
        #if item == 'Grain Mill':
        #    pd_new_data.at[pd_new_data.index[ii],'Grain Mill'] = 1
        #if item == 'Rainwater Harvesting System':
        #    pd_new_data.at[pd_new_data.index[ii],'Rainwater Harvesting System'] = 1
        #if item == 'Water Point':
        #    pd_new_data.at[pd_new_data.index[ii],'Water Point'] = 1
        #if item == 'Borehole':
        #    pd_new_data.at[pd_new_data.index[ii],'Borehole'] = 1
        #if item == 'Kerosene Lantern':
        #    pd_new_data.at[pd_new_data.index[ii],'Kerosene Lantern'] = 1
        #if item == 'Solar Lantern':
        #    pd_new_data.at[pd_new_data.index[ii],'Solar Lantern'] = 1
        #if item == 'Pig':
        #    pd_new_data.at[pd_new_data.index[ii],'Pig'] = 1
        #if item == 'Sugar Cane':
        #    pd_new_data.at[pd_new_data.index[ii],'Sugar Cane'] = 1
        #if item == 'Water Pot':
        #    pd_new_data.at[pd_new_data.index[ii],'Water Pot'] = 1
        #if item == 'Crops (Maize-rice)':
        #    pd_new_data.at[pd_new_data.index[ii],'Crops (Maize-rice)'] = 1
        #if item == 'Candle':
        #    pd_new_data.at[pd_new_data.index[ii],'Candle'] = 1
        #if item == 'Generator':
        #    pd_new_data.at[pd_new_data.index[ii],'Generator'] = 1
        #ii=ii+1
    #i=i+1
      
i=0
j=0
l=len(pd_new_data)
m=len(pd_new_data.columns)
for i in range(l):
    for j in range(m):
        if  pd_new_data.iat[i,j] == 'x': 
            pd_new_data.iat[i,j] = 0
            #print(i, j)  
              
#pd_new_data.to_excel("fff.xlsx")
pd_new_data

Yes, I found the Interview ID column
The column position of Interview ID is 1
There are 26 observations


,Interview ID,Village,Item Name,"lat,long",age,sex,no_people_in_household,highest_grade_completed,distance_fetch_water_one_way,type_of_fuel,water_source,toilet_type
0,60645a4ea2237b3f54690cef,Bwindi,Gas Stove,"-0.9682629102,29.61722332",32.0,1.0,7.0,2.0,1.0,3.0,12.0,2.0
1,606459aea2237b3f546905ed,Gwere,Stove,"3.267508412,33.20886406",26.0,2.0,4.0,1.0,4.0,1.0,1.0,2.0
3,606459b3a2237b3f54690620,Mabwe,Stove,"0.5951310726,30.15312827",32.0,1.0,7.0,2.0,4.0,1.0,1.0,2.0
5,606458d3a2237b3f5468fc80,Okabi,Stove,"3.125250327,30.84880735",29.0,1.0,3.0,3.0,3.0,3.0,11.0,2.0
7,60645a87a2237b3f54690f63,Suam,Gas Stove,"1.227259,34.734948",51.0,1.0,12.0,2.0,4.0,7.0,15.0,2.0
10,606458c6a2237b3f5468fbf2,Bwindi,Stove,"-0.9682629102,29.61722332",34.0,2.0,6.0,2.0,2.0,7.0,2.0,2.0
15,60645ac6a2237b3f5469122c,Bwindi,Stove,"-0.9682629102,29.61722332",75.0,1.0,2.0,0.0,3.0,1.0,11.0,2.0
16,60645abca2237b3f546911be,Okabi,Stove,"3.125250327,30.84880735",49.0,2.0,5.0,1.0,4.0,1.0,11.0,2.0
19,60645ae5a2237b3f54691399,Suam,Gas Stove,"1.227259,34.734948",45.0,2.0,6.0,1.0,3.0,1.0,9.0,2.0
21,60645ae0a2237b3f5469135f,Bihondo,Stove,"0.5785313656,30.1705658",NaN,2.0,2.0,1.0,3.0,1.0,2.0,2.0


In [8]:
i=0
ii=0
iii=0
while ii < len(df_di_selected3.index):    
    if pd_new_data.at[pd_new_data.index.values[i], 'Interview ID'] == df_di_selected3.loc[df_di_selected3.index.values[ii],'Interview ID']: 
        pd_new_data.at[pd_new_data.index.values[i], iii] = df_di_selected3.loc[df_di_selected3.index.values[ii],'Val_1']
        iii=iii+1
        
        empty2 = pd.isnull(df_di_selected3['Val_2'].iloc[ii])
        if empty2 == False:
            pd_new_data.at[pd_new_data.index.values[i], iii] = df_di_selected3.loc[df_di_selected3.index.values[ii],'Val_2']
            iii=iii+1
            
        empty3 = pd.isnull(df_di_selected3['Val_3'].iloc[ii])
        if empty3 == False:
            pd_new_data.at[pd_new_data.index.values[i], iii] = df_di_selected3.loc[df_di_selected3.index.values[ii],'Val_3']
            iii=iii+1
            
        empty4 = pd.isnull(df_di_selected3['Val_4'].iloc[ii])
        if empty4 == False:
            pd_new_data.at[pd_new_data.index.values[i], iii] = df_di_selected3.loc[df_di_selected3.index.values[ii],'Val_4']
            iii=iii+1
            
        empty5 = pd.isnull(df_di_selected3['Val_5'].iloc[ii])
        if empty5 == False:
            pd_new_data.at[pd_new_data.index.values[i], iii] = df_di_selected3.loc[df_di_selected3.index.values[ii],'Val_5']
            iii=iii+1
            
        empty6 = pd.isnull(df_di_selected3['Val_6'].iloc[ii])
        if empty6 == False:
            pd_new_data.at[pd_new_data.index.values[i], iii] = df_di_selected3.loc[df_di_selected3.index.values[ii],'Val_6']
            iii=iii+1
            
        empty7 = pd.isnull(df_di_selected3['Val_7'].iloc[ii])
        if empty7 == False:
            pd_new_data.at[pd_new_data.index.values[i], iii] = df_di_selected3.loc[df_di_selected3.index.values[ii],'Val_7']
            iii=iii+1
            
        empty8 = pd.isnull(df_di_selected3['Val_8'].iloc[ii])
        if empty8 == False:
            pd_new_data.at[pd_new_data.index.values[i], iii] = df_di_selected3.loc[df_di_selected3.index.values[ii],'Val_8']
            iii=iii+1
            
        empty9 = pd.isnull(df_di_selected3['Val_9'].iloc[ii])
        if empty9 == False:
            pd_new_data.at[pd_new_data.index.values[i], iii] = df_di_selected3.loc[df_di_selected3.index.values[ii],'Val_9']
            iii=iii+1
            
        empty10 = pd.isnull(df_di_selected3['Val_10'].iloc[ii])
        if empty10 == False:
            pd_new_data.at[pd_new_data.index.values[i], iii] = df_di_selected3.loc[df_di_selected3.index.values[ii],'Val_10']
            iii=iii+1
            
        empty11 = pd.isnull(df_di_selected3['Val_11'].iloc[ii])
        if empty11 == False:
            pd_new_data.at[pd_new_data.index.values[i], iii] = df_di_selected3.loc[df_di_selected3.index.values[ii],'Val_11']
            iii=iii+1
            
        empty12 = pd.isnull(df_di_selected3['Val_12'].iloc[ii])
        if empty12 == False:
            pd_new_data.at[pd_new_data.index.values[i], iii] = df_di_selected3.loc[df_di_selected3.index.values[ii],'Val_12']
            iii=iii+1
            
        empty13 = pd.isnull(df_di_selected3['Val_13'].iloc[ii])
        if empty13 == False:
            pd_new_data.at[pd_new_data.index.values[i], iii] = df_di_selected3.loc[df_di_selected3.index.values[ii],'Val_13']
            iii=iii+1
            
        empty14 = pd.isnull(df_di_selected3['Val_14'].iloc[ii])
        if empty14 == False:
            pd_new_data.at[pd_new_data.index.values[i], iii] = df_di_selected3.loc[df_di_selected3.index.values[ii],'Val_14']
            iii=iii+1
            
    if pd_new_data.at[pd_new_data.index.values[i], 'Interview ID'] != df_di_selected3.loc[df_di_selected3.index.values[ii],'Interview ID']:
        i=i+1 
        iii=0
    else: 
        ii=ii+1 
        
    #print(i,ii,iii)

pd_new_data

,Interview ID,Village,Item Name,"lat,long",age,sex,no_people_in_household,highest_grade_completed,distance_fetch_water_one_way,type_of_fuel,water_source,toilet_type,0,1,2,3,4,5,6,7
0,60645a4ea2237b3f54690cef,Bwindi,Gas Stove,"-0.9682629102,29.61722332",32.0,1.0,7.0,2.0,1.0,3.0,12.0,2.0,Time benefit,Aspiration,NaN,NaN,NaN,NaN,NaN,NaN
1,606459aea2237b3f546905ed,Gwere,Stove,"3.267508412,33.20886406",26.0,2.0,4.0,1.0,4.0,1.0,1.0,2.0,Being healthy,Medical treatment,Comfort,Water Security,NaN,NaN,NaN,NaN
3,606459b3a2237b3f54690620,Mabwe,Stove,"0.5951310726,30.15312827",32.0,1.0,7.0,2.0,4.0,1.0,1.0,2.0,Food Security,Food Security,Being healthy,NaN,NaN,NaN,NaN,NaN
5,606458d3a2237b3f5468fc80,Okabi,Stove,"3.125250327,30.84880735",29.0,1.0,3.0,3.0,3.0,3.0,11.0,2.0,Fixed costs,Accessibility to services,Fixed costs,Being healthy,NaN,NaN,NaN,NaN
7,60645a87a2237b3f54690f63,Suam,Gas Stove,"1.227259,34.734948",51.0,1.0,12.0,2.0,4.0,7.0,15.0,2.0,Accessibility to services,Preservation of natural environment,Availability of goods/products,Accessibility to services,Time benefit,Unburden,Being healthy,NaN
10,606458c6a2237b3f5468fbf2,Bwindi,Stove,"-0.9682629102,29.61722332",34.0,2.0,6.0,2.0,2.0,7.0,2.0,2.0,Time management,Accessibility to services,Fixed costs,Accessibility to services,Availability of goods/products,Time benefit,Time benefit,Availability of goods/products
15,60645ac6a2237b3f5469122c,Bwindi,Stove,"-0.9682629102,29.61722332",75.0,1.0,2.0,0.0,3.0,1.0,11.0,2.0,Time benefit,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,60645abca2237b3f546911be,Okabi,Stove,"3.125250327,30.84880735",49.0,2.0,5.0,1.0,4.0,1.0,11.0,2.0,Time benefit,Availability of goods/products,Unburden,NaN,NaN,NaN,NaN,NaN
19,60645ae5a2237b3f54691399,Suam,Gas Stove,"1.227259,34.734948",45.0,2.0,6.0,1.0,3.0,1.0,9.0,2.0,Time benefit,Unburden,Being healthy,Time benefit,Income,NaN,NaN,NaN
21,60645ae0a2237b3f5469135f,Bihondo,Stove,"0.5785313656,30.1705658",NaN,2.0,2.0,1.0,3.0,1.0,2.0,2.0,Food Security,Being healthy,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Here, we include the observations that have the required values but have not been chosen before. 

df_di_selected_all = pd.DataFrame()

for column in df_di_selected2: 
    if column == 'Interview ID':
        i=0
        print('Yes, I found the', column, 'column')
        j = df_di_selected2.columns.get_loc(column)
        print('The column position of', column, 'is', j)
        df_di_selected_all.at[i, column] = df_di_selected2.loc[i,'Interview ID']
        i=i+1
        for row in df_di_selected2.iterrows():
            if df_di_selected2.loc[i,'Interview ID']!=df_di_selected2.loc[i-1,'Interview ID']:
                df_di_selected_all.at[i, column] = df_di_selected2.loc[i,'Interview ID'] 
            i=i+1
            if i == len(df_di_selected2.index)-1:
                break

unique_values_ID = pd.unique(df_di_selected_all['Interview ID'])
print("There are", len(unique_values_ID), "observations")

#Matching the Village with the interviewer

i=0
for row in df_di_selected_all.iterrows():
    df_di_selected_all.at[df_di_selected_all.index.values[i], 'Village'] = df_di_selected2.loc[df_di_selected_all.index.values[i],'Village'] 
    i=i+1
    if i == len(df_di_selected2.index)-1:
        break
        
#Matching the Item Name with the interviewer

i=0
for row in df_di_selected_all.iterrows():
    df_di_selected_all.at[df_di_selected_all.index.values[i], 'Item Name'] = df_di_selected2.loc[df_di_selected_all.index.values[i], 'Item Name'] 
    i=i+1
    if i == len(df_di_selected2.index)-1:
        break

#Matching the latitude and longitude with the interviewer

i=0
for row in df_di_selected_all.iterrows():
    df_di_selected_all.at[df_di_selected_all.index.values[i], 'lat,long'] = df_di_selected2.loc[df_di_selected_all.index.values[i],'lat,long'] 
    i=i+1
    if i == len(df_di_selected2.index)-1:
        break

#Matching the age with the interviewer

i=0
for row in df_di_selected_all.iterrows():
    df_di_selected_all.at[df_di_selected_all.index.values[i], 'age'] = df_di_selected2.loc[df_di_selected_all.index.values[i],'age'] 
    i=i+1
    if i == len(df_di_selected2.index)-1:
        break        
        
#Matching the sex with the interviewer

i=0
for row in df_di_selected_all.iterrows():
    df_di_selected_all.at[df_di_selected_all.index.values[i], 'sex'] = df_di_selected2.loc[df_di_selected_all.index.values[i],'sex'] 
    i=i+1
    if i == len(df_di_selected2.index)-1:
        break     
        
#Matching the number of people in household with the interviewer

i=0
for row in df_di_selected_all.iterrows():
    df_di_selected_all.at[df_di_selected_all.index.values[i], 'no_people_in_household'] = df_di_selected2.loc[df_di_selected_all.index.values[i],'no_people_in_household'] 
    i=i+1
    if i == len(df_di_selected2.index)-1:
        break  

#Matching the highest grade completed with the interviewer

i=0
for row in df_di_selected_all.iterrows():
    df_di_selected_all.at[df_di_selected_all.index.values[i], 'highest_grade_completed'] = df_di_selected2.loc[df_di_selected_all.index.values[i],'highest_grade_completed'] 
    i=i+1
    if i == len(df_di_selected2.index)-1:
        break        

#Matching the distance to water source with the interviewer

i=0
for row in df_di_selected_all.iterrows():
    df_di_selected_all.at[df_di_selected_all.index.values[i], 'distance_fetch_water_one_way'] = df_di_selected2.loc[df_di_selected_all.index.values[i],'distance_fetch_water_one_way'] 
    i=i+1
    if i == len(df_di_selected2.index)-1:
        break   
        
#Matching the type of fuel with the interviewer

i=0
for row in df_di_selected_all.iterrows():
    if df_di_selected2.loc[df_di_selected_all.index.values[i],'cooking_energy_source'] == '1;2':
        df_di_selected_all.at[df_di_selected_all.index.values[i], 'type_of_fuel'] = 6
    elif df_di_selected2.loc[df_di_selected_all.index.values[i],'cooking_energy_source'] == '1;3':
        df_di_selected_all.at[df_di_selected_all.index.values[i], 'type_of_fuel'] = 7
    else:
        df_di_selected_all.at[df_di_selected_all.index.values[i], 'type_of_fuel'] = df_di_selected2.loc[df_di_selected_all.index.values[i],'cooking_energy_source'] 
    i=i+1
    if i == len(df_di_selected2.index)-1:
        break

#Matching the source of water with the interviewer

i=0
for row in df_di_selected_all.iterrows():
    if df_di_selected2.loc[df_di_selected_all.index.values[i],'water_source'] == '6;11':
        df_di_selected_all.at[df_di_selected_all.index.values[i], 'water_source'] = 13
    elif df_di_selected2.loc[df_di_selected_all.index.values[i],'water_source'] == '9;11':
        df_di_selected_all.at[df_di_selected_all.index.values[i], 'water_source'] = 14
    elif df_di_selected2.loc[df_di_selected_all.index.values[i],'water_source'] == '1;9':
        df_di_selected_all.at[df_di_selected_all.index.values[i], 'water_source'] = 15
    elif df_di_selected2.loc[df_di_selected_all.index.values[i],'water_source'] == '1;7':
        df_di_selected_all.at[df_di_selected_all.index.values[i], 'water_source'] = 16
    elif df_di_selected2.loc[df_di_selected_all.index.values[i],'water_source'] == '4;12':
        df_di_selected_all.at[df_di_selected_all.index.values[i], 'water_source'] = 17
    elif df_di_selected2.loc[df_di_selected_all.index.values[i],'water_source'] == '3;4':
        df_di_selected_all.at[df_di_selected_all.index.values[i], 'water_source'] = 18
    elif df_di_selected2.loc[df_di_selected_all.index.values[i],'water_source'] == '7;9':
        df_di_selected_all.at[df_di_selected_all.index.values[i], 'water_source'] = 19
    elif df_di_selected2.loc[df_di_selected_all.index.values[i],'water_source'] == '6;9;11':
        df_di_selected_all.at[df_di_selected_all.index.values[i], 'water_source'] = 20
    elif df_di_selected2.loc[df_di_selected_all.index.values[i],'water_source'] == '4;9':
        df_di_selected_all.at[df_di_selected_all.index.values[i], 'water_source'] = 21
    else:
        df_di_selected_all.at[df_di_selected_all.index.values[i], 'water_source'] = df_di_selected2.loc[df_di_selected_all.index.values[i],'water_source'] 
    i=i+1
    if i == len(df_di_selected2.index)-1:
        break  
        
#Matching the type of toilet with the interviewer

i=0
for row in df_di_selected_all.iterrows():
    if df_di_selected2.loc[df_di_selected_all.index.values[i],'toilet_type'] == '1;2':
        df_di_selected_all.at[df_di_selected_all.index.values[i], 'toilet_type'] = 6
    else:    
        df_di_selected_all.at[df_di_selected_all.index.values[i], 'toilet_type'] = df_di_selected2.loc[df_di_selected_all.index.values[i],'toilet_type'] 
    i=i+1
    if i == len(df_di_selected2.index)-1:
        break
  
              
#df_di_selected_all.to_excel("df_di_selected_all.xlsx")
df_di_selected_all

Yes, I found the Interview ID column
The column position of Interview ID is 0
There are 200 observations


,Interview ID,Village,Item Name,"lat,long",age,sex,no_people_in_household,highest_grade_completed,distance_fetch_water_one_way,type_of_fuel,water_source,toilet_type
0,60645a53a2237b3f54690d25,Lama,Hoe,"3.668252174,31.92603986",24.0,2.0,3.0,1.0,2.0,1.0,6.0,2.0
58,60645a4ea2237b3f54690cef,Bwindi,Pig,"-0.9682629102,29.61722332",32.0,1.0,7.0,2.0,1.0,3.0,12.0,2.0
90,60645af1a2237b3f54691429,Bwindi,Pencil,"-0.9682629102,29.61722332",39.0,2.0,7.0,4.0,1.0,7.0,2.0,6.0
135,606459c7a2237b3f546906fa,Okabi,Crops (Maize-rice),"3.125250327,30.84880735",60.0,1.0,3.0,0.0,3.0,1.0,13.0,2.0
172,60645877a2237b3f5468f898,Bihondo,Bike,"0.5785313656,30.1705658",29.0,1.0,5.0,2.0,5.0,1.0,14.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...
6100,60d06f82defc3909beb589e7,Kyenjojo town,Tree,"0.6093,30.6401",52.0,1.0,7.0,3.0,5.0,1.0,3.0,2.0
6136,60d06f82defc3909beb589e8,Katooke,Motorcycle,"0.6093,30.6401",42.0,1.0,4.0,3.0,5.0,1.0,3.0,2.0
6162,60d06f83defc3909beb589e9,Katumba,Solar PV System,"0.5897,30.2549",37.0,2.0,5.0,1.0,5.0,1.0,3.0,1.0
6185,60d06f83defc3909beb589ea,Katumba,Motorcycle,"0.5897,30.2549",41.0,1.0,6.0,4.0,5.0,1.0,3.0,2.0


In [10]:
i=0
ii=0
iii=0
while ii < len(df_di_selected2.index):    
    if df_di_selected_all.at[df_di_selected_all.index.values[i], 'Interview ID'] == df_di_selected2.loc[df_di_selected2.index.values[ii],'Interview ID']: 
        df_di_selected_all.at[df_di_selected_all.index.values[i], iii] = df_di_selected2.loc[df_di_selected2.index.values[ii],'Val_1']
        iii=iii+1
        
        empty2 = pd.isnull(df_di_selected2['Val_2'].iloc[ii])
        if empty2 == False:
            df_di_selected_all.at[df_di_selected_all.index.values[i], iii] = df_di_selected2.loc[df_di_selected2.index.values[ii],'Val_2']
            iii=iii+1

        empty3 = pd.isnull(df_di_selected2['Val_3'].iloc[ii])
        if empty3 == False:
            df_di_selected_all.at[df_di_selected_all.index.values[i], iii] = df_di_selected2.loc[df_di_selected2.index.values[ii],'Val_3']
            iii=iii+1
            
        empty4 = pd.isnull(df_di_selected2['Val_4'].iloc[ii])
        if empty4 == False:
            df_di_selected_all.at[df_di_selected_all.index.values[i], iii] = df_di_selected2.loc[df_di_selected2.index.values[ii],'Val_4']
            iii=iii+1
            
        empty5 = pd.isnull(df_di_selected2['Val_5'].iloc[ii])
        if empty5 == False:
            df_di_selected_all.at[df_di_selected_all.index.values[i], iii] = df_di_selected2.loc[df_di_selected2.index.values[ii],'Val_5']
            iii=iii+1
            
        empty6 = pd.isnull(df_di_selected2['Val_6'].iloc[ii])
        if empty6 == False:
            df_di_selected_all.at[df_di_selected_all.index.values[i], iii] = df_di_selected2.loc[df_di_selected2.index.values[ii],'Val_6']
            iii=iii+1
            
        empty7 = pd.isnull(df_di_selected2['Val_7'].iloc[ii])
        if empty7 == False:
            df_di_selected_all.at[df_di_selected_all.index.values[i], iii] = df_di_selected2.loc[df_di_selected2.index.values[ii],'Val_7']
            iii=iii+1
            
        empty8 = pd.isnull(df_di_selected2['Val_8'].iloc[ii])
        if empty8 == False:
            df_di_selected_all.at[df_di_selected_all.index.values[i], iii] = df_di_selected2.loc[df_di_selected2.index.values[ii],'Val_8']
            iii=iii+1
            
        empty9 = pd.isnull(df_di_selected2['Val_9'].iloc[ii])
        if empty9 == False:
            df_di_selected_all.at[df_di_selected_all.index.values[i], iii] = df_di_selected2.loc[df_di_selected2.index.values[ii],'Val_9']
            iii=iii+1
            
        empty10 = pd.isnull(df_di_selected2['Val_10'].iloc[ii])
        if empty10 == False:
            df_di_selected_all.at[df_di_selected_all.index.values[i], iii] = df_di_selected2.loc[df_di_selected2.index.values[ii],'Val_10']
            iii=iii+1
            
        empty11 = pd.isnull(df_di_selected2['Val_11'].iloc[ii])
        if empty11 == False:
            df_di_selected_all.at[df_di_selected_all.index.values[i], iii] = df_di_selected2.loc[df_di_selected2.index.values[ii],'Val_11']
            iii=iii+1
            
        empty12 = pd.isnull(df_di_selected2['Val_12'].iloc[ii])
        if empty12 == False:
            df_di_selected_all.at[df_di_selected_all.index.values[i], iii] = df_di_selected2.loc[df_di_selected2.index.values[ii],'Val_12']
            iii=iii+1
            
        empty13 = pd.isnull(df_di_selected2['Val_13'].iloc[ii])
        if empty13 == False:
            df_di_selected_all.at[df_di_selected_all.index.values[i], iii] = df_di_selected2.loc[df_di_selected2.index.values[ii],'Val_13']
            iii=iii+1
            
        empty14 = pd.isnull(df_di_selected2['Val_14'].iloc[ii])
        if empty14 == False:
            df_di_selected_all.at[df_di_selected_all.index.values[i], iii] = df_di_selected2.loc[df_di_selected2.index.values[ii],'Val_14']
            iii=iii+1
            
    if df_di_selected_all.at[df_di_selected_all.index.values[i], 'Interview ID'] != df_di_selected2.loc[df_di_selected2.index.values[ii],'Interview ID']:
        i=i+1 
        iii=0
    else: 
        ii=ii+1 
        
    #print(i,ii,iii)

df_di_selected_all.to_excel("df_di_selected_all.xlsx")
df_di_selected_all = df_di_selected_all.reset_index()
df_di_selected_all

,index,Interview ID,Village,Item Name,"lat,long",age,sex,no_people_in_household,highest_grade_completed,distance_fetch_water_one_way,...,119,120,121,122,123,124,125,126,127,128
0,0,60645a53a2237b3f54690d25,Lama,Hoe,"3.668252174,31.92603986",24.0,2.0,3.0,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,58,60645a4ea2237b3f54690cef,Bwindi,Pig,"-0.9682629102,29.61722332",32.0,1.0,7.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,90,60645af1a2237b3f54691429,Bwindi,Pencil,"-0.9682629102,29.61722332",39.0,2.0,7.0,4.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,135,606459c7a2237b3f546906fa,Okabi,Crops (Maize-rice),"3.125250327,30.84880735",60.0,1.0,3.0,0.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,172,60645877a2237b3f5468f898,Bihondo,Bike,"0.5785313656,30.1705658",29.0,1.0,5.0,2.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,6100,60d06f82defc3909beb589e7,Kyenjojo town,Tree,"0.6093,30.6401",52.0,1.0,7.0,3.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
196,6136,60d06f82defc3909beb589e8,Katooke,Motorcycle,"0.6093,30.6401",42.0,1.0,4.0,3.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
197,6162,60d06f83defc3909beb589e9,Katumba,Solar PV System,"0.5897,30.2549",37.0,2.0,5.0,1.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198,6185,60d06f83defc3909beb589ea,Katumba,Motorcycle,"0.5897,30.2549",41.0,1.0,6.0,4.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
chosen_values = ['Food Security', 'Time benefit', 'Being healthy', 'Accessibility to services', 'Fixed costs']
chosen_values
len(chosen_values)

df_di_selected_extra = pd.DataFrame()

i = 0
ii = 0
iii= 0
while i < len(df_di_selected_all.index):  
    if df_di_selected_all.loc[df_di_selected_all.index.values[i], iii] in chosen_values:
        get_interview_id = df_di_selected_all.loc[df_di_selected_all.index.values[i], 'Interview ID']
        if (df_di_selected3['Interview ID'].eq(get_interview_id)).any() == False:
            df_di_selected_extra=df_di_selected_extra.append(df_di_selected_all.loc[[i]])                 
        else:
            pass
        i=i+1
    else:
        i=i+1

i = 0
ii = 0
iii= 1

while iii < len(range(0,128)):
    i=0
    while i < len(df_di_selected_all.index):  
        if df_di_selected_all.loc[df_di_selected_all.index.values[i], iii] in chosen_values:
            get_interview_id = df_di_selected_all.loc[df_di_selected_all.index.values[i], 'Interview ID']
            if (df_di_selected3['Interview ID'].eq(get_interview_id)).any() == False and (df_di_selected_extra['Interview ID'].eq(get_interview_id)).any() == False:
                df_di_selected_extra=df_di_selected_extra.append(df_di_selected_all.loc[[i]])        
            else:
                pass
            i=i+1
        else:
            i=i+1
    iii=iii+1
del df_di_selected_extra['index']
#df_di_selected_extra.to_excel("df_di_selected_extra.xlsx")
df_di_selected_extra

,Interview ID,Village,Item Name,"lat,long",age,sex,no_people_in_household,highest_grade_completed,distance_fetch_water_one_way,type_of_fuel,...,119,120,121,122,123,124,125,126,127,128
0,60645a53a2237b3f54690d25,Lama,Hoe,"3.668252174,31.92603986",24.0,2.0,3.0,1.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,60645a36a2237b3f54690be5,Mabwe,Goat,"0.5951310726,30.15312827",35.0,2.0,3.0,2.0,4.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,60645b0ba2237b3f5469153b,Bwindi,Bike,"-0.9682629102,29.61722332",35.0,2.0,8.0,3.0,2.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,6064590ea2237b3f5468fefa,Bihondo,Motorcycle,"0.5785313656,30.1705658",62.0,1.0,14.0,1.0,4.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,60645ad6a2237b3f546912eb,Bwindi,Toothbrush,"-0.9682629102,29.61722332",72.0,1.0,8.0,0.0,1.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,60d06f50defc3909beb5899f,Pece Cubu,Radio,"2.7724,32.2881",54.0,1.0,13.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119,60d06f4edefc3909beb5899b,Kanyansi,Quran,"1.0528,30.5401",34.0,1.0,5.0,2.0,6.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
122,60d06f4fdefc3909beb5899e,Kanyansi,Fridge,"1.0528,30.5401",37.0,1.0,4.0,2.0,6.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193,60d06f81defc3909beb589e5,Mugooma,Mobile Phone,"0.6546,30.2801",41.0,2.0,9.0,2.0,6.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df_di_selected_extra['Annotations2'] = df_di_selected_extra[[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,
                                                             24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,
                                                             45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,
                                                            65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,
                                                             86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,
                                                             105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,
                                                             120,121,122,123,124,125,126,127,128]].apply(lambda x: ' '.join(x[x.notnull()]), axis = 1)
df_di_selected_extra

,Interview ID,Village,Item Name,"lat,long",age,sex,no_people_in_household,highest_grade_completed,distance_fetch_water_one_way,type_of_fuel,...,120,121,122,123,124,125,126,127,128,Annotations2
0,60645a53a2237b3f54690d25,Lama,Hoe,"3.668252174,31.92603986",24.0,2.0,3.0,1.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Being healthy Unburden Income Fixed costs Inco...
7,60645a36a2237b3f54690be5,Mabwe,Goat,"0.5951310726,30.15312827",35.0,2.0,3.0,2.0,4.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Food Security Being healthy Food Security Inco...
11,60645b0ba2237b3f5469153b,Bwindi,Bike,"-0.9682629102,29.61722332",35.0,2.0,8.0,3.0,2.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Accessibility to services Travel (of people) I...
33,6064590ea2237b3f5468fefa,Bihondo,Motorcycle,"0.5785313656,30.1705658",62.0,1.0,14.0,1.0,4.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Accessibility to services Time benefit Travel ...
36,60645ad6a2237b3f546912eb,Bwindi,Toothbrush,"-0.9682629102,29.61722332",72.0,1.0,8.0,0.0,1.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Being healthy Faith Morality Personal performa...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,60d06f50defc3909beb5899f,Pece Cubu,Radio,"2.7724,32.2881",54.0,1.0,13.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Being informed Entertainment Being informed Be...
119,60d06f4edefc3909beb5899b,Kanyansi,Quran,"1.0528,30.5401",34.0,1.0,5.0,2.0,6.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Faith Mental health Faith Faith Asset Income A...
122,60d06f4fdefc3909beb5899e,Kanyansi,Fridge,"1.0528,30.5401",37.0,1.0,4.0,2.0,6.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Travel (of people) Home (feeling) Personal per...
193,60d06f81defc3909beb589e5,Mugooma,Mobile Phone,"0.6546,30.2801",41.0,2.0,9.0,2.0,6.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Connection Continuous costs Connection Persona...


In [13]:
#Here, we only select the observations that have chosen the first 5 top values
df = df_di_selected_extra
kw = ['Food Security', 'Time benefit', 'Being healthy', 'Accessibility to services', 'Fixed costs']
a = 'Food Security'
b = 'Time benefit'
c = 'Being healthy'
d = 'Accessibility to services'
e = 'Fixed costs'

ind=[]
count = 0
for idx, value in df.iterrows():
    #print(idx)
    x = 0
    #for u in kw:
    if (a and b and c and d and e) in value['Annotations2']:
        #print(idx)
        ind.append(True)
        count=count+1
        x = 1
    if x == 0:
        ind.append(False)
        #print(ind)

df['flag'] = ind
display(count)
df.reset_index(inplace=True)
df

117

,index,Interview ID,Village,Item Name,"lat,long",age,sex,no_people_in_household,highest_grade_completed,distance_fetch_water_one_way,...,121,122,123,124,125,126,127,128,Annotations2,flag
0,0,60645a53a2237b3f54690d25,Lama,Hoe,"3.668252174,31.92603986",24.0,2.0,3.0,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Being healthy Unburden Income Fixed costs Inco...,True
1,7,60645a36a2237b3f54690be5,Mabwe,Goat,"0.5951310726,30.15312827",35.0,2.0,3.0,2.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Food Security Being healthy Food Security Inco...,False
2,11,60645b0ba2237b3f5469153b,Bwindi,Bike,"-0.9682629102,29.61722332",35.0,2.0,8.0,3.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Accessibility to services Travel (of people) I...,True
3,33,6064590ea2237b3f5468fefa,Bihondo,Motorcycle,"0.5785313656,30.1705658",62.0,1.0,14.0,1.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Accessibility to services Time benefit Travel ...,True
4,36,60645ad6a2237b3f546912eb,Bwindi,Toothbrush,"-0.9682629102,29.61722332",72.0,1.0,8.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Being healthy Faith Morality Personal performa...,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,123,60d06f50defc3909beb5899f,Pece Cubu,Radio,"2.7724,32.2881",54.0,1.0,13.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Being informed Entertainment Being informed Be...,False
170,119,60d06f4edefc3909beb5899b,Kanyansi,Quran,"1.0528,30.5401",34.0,1.0,5.0,2.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Faith Mental health Faith Faith Asset Income A...,True
171,122,60d06f4fdefc3909beb5899e,Kanyansi,Fridge,"1.0528,30.5401",37.0,1.0,4.0,2.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Travel (of people) Home (feeling) Personal per...,True
172,193,60d06f81defc3909beb589e5,Mugooma,Mobile Phone,"0.6546,30.2801",41.0,2.0,9.0,2.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Connection Continuous costs Connection Persona...,True


In [14]:

df2=df
i=0
while i < len(df_di_selected_extra):
    if df.loc[df.index.values[i],'flag'] == True:
        df2 = df2.drop(i)
        i=i+1
    else:
        i=i+1
df2

,index,Interview ID,Village,Item Name,"lat,long",age,sex,no_people_in_household,highest_grade_completed,distance_fetch_water_one_way,...,121,122,123,124,125,126,127,128,Annotations2,flag
1,7,60645a36a2237b3f54690be5,Mabwe,Goat,"0.5951310726,30.15312827",35.0,2.0,3.0,2.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Food Security Being healthy Food Security Inco...,False
8,63,60645af8a2237b3f5469146c,Bihondo,Hoe,"0.5785313656,30.1705658",45.0,1.0,7.0,0.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Food Security Food Security Food Security Bein...,False
9,64,60645921a2237b3f5468ffc1,Okabi,Solar Lantern,"3.125250327,30.84880735",64.0,2.0,4.0,0.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Accessibility to services Safety (from animals...,False
15,100,60d06f3edefc3909beb58988,Ngoro,Clothes,"0.1966,30.1683",28.0,1.0,4.0,4.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Being healthy Medical treatment Unburden Conti...,False
22,124,60d06f50defc3909beb589a0,Kyigabo,Hoe,"0.2642,30.1084",35.0,1.0,5.0,2.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Food Security Income Income Entrepreneurship E...,False
25,133,60d06f53defc3909beb589a9,Kazinga,Cow,"0.2642,30.1084",31.0,2.0,5.0,0.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Food Security Comfort Caring for others Being ...,False
26,141,60d06f60defc3909beb589b1,Kitumba,Fridge,"0.5897,30.2549",37.0,1.0,4.0,1.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Food Security Being informed Income Togetherne...,False
30,152,60d06f63defc3909beb589bc,Kakanju,Money,"0.5424,30.1965",28.0,2.0,3.0,2.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Food Security Home (feeling) Continuous costs ...,False
31,154,60d06f64defc3909beb589be,Kaburegye,Goat,"0.5424,30.1965",45.0,2.0,4.0,0.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Food Security Being healthy Togetherness (love...,False
35,162,60d06f67defc3909beb589c6,Karambi,Hospital,"0.1699,30.0781",26.0,2.0,3.0,2.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Accessibility to services Medical treatment Av...,False


In [15]:
df_di_selected_extra = df2

In [16]:
#pd_new_data = pd.concat([pd_new_data, df_di_selected_extra], axis=0)
#pd_new_data.reset_index(inplace=True)
pd_new_data = df_di_selected_extra
pd_new_data.to_excel("df_di_selected_falses.xlsx")

In [17]:
#Importing the frecuency of values chosen by the poeple who chose stove, gas stove, school, and hospital

#This is to name the columns only!!!

data_values = pd.DataFrame(pd.read_excel("06._Stephi_List_of_Values_v14.xlsx"))
print("There are", len(data_values), "values in Stephi's data set")
del data_values['Unnamed: 0']
data_values.head(15)

There are 18 values in Stephi's data set


,Values Total,Frecuency,Percent
0,Food Security,16,15.238095
1,Time benefit,15,14.285714
2,Accessibility to services,14,13.333333
3,Fixed costs,11,10.476190
4,Being healthy,10,9.523810
5,Availability of goods/products,8,7.619048
6,Preservation of natural environment,7,6.666667
7,Ease of use,5,4.761905
8,Unburden,4,3.809524
9,Comfort,3,2.857143


In [18]:
#Split the main table into only values table and only geospatial table
df_only_values = pd_new_data[[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,
                              34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,
                              65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,
                              96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,
                              120,121,122,123,124,125,126,127,128]].copy() #This is the number of columns of values
                              #in the last Data Frame
display(df_only_values)
df_only_geo = pd_new_data[['Interview ID', 'Village', 'lat,long','age', 'sex', 'no_people_in_household', 
                           'highest_grade_completed', 'distance_fetch_water_one_way', 
                           'type_of_fuel', 'water_source','toilet_type']].copy()

#print(pd_new_data.loc[pd_new_data.index.values[0],19])

#Creation of a new DF for placing with a 'yes' or 'no' the chose of a value for each observation 

df_new_data_order = pd.DataFrame('x', index=range(len(df_only_values)), columns=list(data_values['Values Total'].tolist()))
print('Max Values Listed:', len(df_only_values.columns),'Observations:', (len(df_only_values)),'Total Values:', len(df_new_data_order.columns))   
display(df_new_data_order)

i=0
j=0
k=0
while i < len(df_only_values):
    #print('soyi', i)
    j=0
    while j < len(df_only_values.columns):
        #print('-soyj', j)
        k=0
        while k < len(df_new_data_order.columns):
            #print('--soyk', k)
            #print(df_new_data_order.columns[0])
            if df_only_values.loc[df_only_values.index.values[i], j] == df_new_data_order.columns[k]:
                #print('encontrado', df_only_values.loc[df_only_values.index.values[i], j], i, j, k)
                #df_new_data_order.loc[df_new_data_order.index.values[i], j] = 'yes'
                df_new_data_order.iat[i,k] = 1
                #print('ok')
                break
            else:
                k=k+1
        j=j+1        
    i=i+1       

df_new_data_order

,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,128
1,Food Security,Being healthy,Food Security,Income,Yield,Travel (of people),Unburden,Travel (of people),Travel (of people),Travel (of people),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Food Security,Food Security,Food Security,Being healthy,Knowledge attainment (learning),Togetherness (loved ones),Being healthy,Income,Continuous costs,Tradition,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Accessibility to services,"Safety (from animals, items, nature)",Accessibility to services,"Safety (from animals, items, nature)","Safety (from animals, items, nature)",Being healthy,Accessibility to services,Aspiration,Medical treatment,Income,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,Being healthy,Medical treatment,Unburden,Continuous costs,Home (feeling),Personal performance,Togetherness (loved ones),Availability of goods/products,Income,Freedom of choice,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,Food Security,Income,Income,Entrepreneurship,Ease of use,Income,Togetherness (loved ones),Being informed,Income,Unburden,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,Food Security,Comfort,Caring for others,Being healthy,Togetherness (loved ones),Recreation,Income,Togetherness (loved ones),Connection,Ease of use,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,Food Security,Being informed,Income,Togetherness (loved ones),Personal performance,Ease of use,Comfort,Unburden,Aesthetics,Entertainment,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,Food Security,Home (feeling),Continuous costs,Continuous costs,Togetherness (loved ones),Food Security,Income,Ease of use,Self-worth,Respect from others,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,Food Security,Being healthy,Togetherness (loved ones),Personal performance,Knowledge attainment (learning),Income,Unburden,Togetherness (loved ones),Food Security,Shelter,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,Accessibility to services,Medical treatment,Availability of goods/products,Being healthy,Water Security,Accessibility to services,Medical treatment,Availability of goods/products,Being healthy,Water Security,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Max Values Listed: 129 Observations: 57 Total Values: 18


,Food Security,Time benefit,Accessibility to services,Fixed costs,Being healthy,Availability of goods/products,Preservation of natural environment,Ease of use,Unburden,Comfort,Water Security,Medical treatment,Appeal (senses),Personal performance,Aspiration,Time management,Income,Reliability
0,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x
1,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x
2,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x
3,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x
4,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x
5,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x
6,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x
7,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x
8,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x
9,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x


,Food Security,Time benefit,Accessibility to services,Fixed costs,Being healthy,Availability of goods/products,Preservation of natural environment,Ease of use,Unburden,Comfort,Water Security,Medical treatment,Appeal (senses),Personal performance,Aspiration,Time management,Income,Reliability
0,1,1,1,x,1,x,x,x,1,1,x,x,1,x,1,x,1,1
1,1,1,1,x,1,x,x,x,x,1,1,1,1,1,1,x,1,x
2,1,x,1,x,1,1,x,1,x,1,x,1,x,1,1,x,1,x
3,1,x,x,x,1,1,x,x,1,x,1,1,x,1,x,x,1,x
4,1,x,1,x,x,1,x,1,1,1,x,x,x,1,x,1,1,x
5,1,x,1,x,1,1,x,1,x,1,x,x,x,1,x,x,1,x
6,1,x,1,x,1,1,x,1,1,1,1,1,x,1,x,x,1,1
7,1,x,x,x,1,x,x,1,x,x,x,x,x,1,1,x,1,x
8,1,x,1,x,1,1,x,1,1,x,1,1,x,1,x,x,1,x
9,1,1,1,x,1,1,x,1,1,1,1,1,x,1,x,x,1,1


In [19]:
df_new_data_order

,Food Security,Time benefit,Accessibility to services,Fixed costs,Being healthy,Availability of goods/products,Preservation of natural environment,Ease of use,Unburden,Comfort,Water Security,Medical treatment,Appeal (senses),Personal performance,Aspiration,Time management,Income,Reliability
0,1,1,1,x,1,x,x,x,1,1,x,x,1,x,1,x,1,1
1,1,1,1,x,1,x,x,x,x,1,1,1,1,1,1,x,1,x
2,1,x,1,x,1,1,x,1,x,1,x,1,x,1,1,x,1,x
3,1,x,x,x,1,1,x,x,1,x,1,1,x,1,x,x,1,x
4,1,x,1,x,x,1,x,1,1,1,x,x,x,1,x,1,1,x
5,1,x,1,x,1,1,x,1,x,1,x,x,x,1,x,x,1,x
6,1,x,1,x,1,1,x,1,1,1,1,1,x,1,x,x,1,1
7,1,x,x,x,1,x,x,1,x,x,x,x,x,1,1,x,1,x
8,1,x,1,x,1,1,x,1,1,x,1,1,x,1,x,x,1,x
9,1,1,1,x,1,1,x,1,1,1,1,1,x,1,x,x,1,1


In [20]:
i=0
j=0
l=len(df_new_data_order)
m=len(df_new_data_order.columns)
for i in range(l):
    for j in range(m):
        if  df_new_data_order.iat[i,j] == 'x': 
            df_new_data_order.iat[i,j] = 0
            #print(i, j)
#df_new_data_order.to_excel("v3_yes_no_values.xlsx") 
df_new_data_order

,Food Security,Time benefit,Accessibility to services,Fixed costs,Being healthy,Availability of goods/products,Preservation of natural environment,Ease of use,Unburden,Comfort,Water Security,Medical treatment,Appeal (senses),Personal performance,Aspiration,Time management,Income,Reliability
0,1,1,1,0,1,0,0,0,1,1,0,0,1,0,1,0,1,1
1,1,1,1,0,1,0,0,0,0,1,1,1,1,1,1,0,1,0
2,1,0,1,0,1,1,0,1,0,1,0,1,0,1,1,0,1,0
3,1,0,0,0,1,1,0,0,1,0,1,1,0,1,0,0,1,0
4,1,0,1,0,0,1,0,1,1,1,0,0,0,1,0,1,1,0
5,1,0,1,0,1,1,0,1,0,1,0,0,0,1,0,0,1,0
6,1,0,1,0,1,1,0,1,1,1,1,1,0,1,0,0,1,1
7,1,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0,1,0
8,1,0,1,0,1,1,0,1,1,0,1,1,0,1,0,0,1,0
9,1,1,1,0,1,1,0,1,1,1,1,1,0,1,0,0,1,1


In [21]:
#Assign new indices to the 'df_only_geo' table to be then merged witht the 'df_new_data_order' table
df_only_geo.reset_index(inplace=True)
df_only_geo

,index,Interview ID,Village,"lat,long",age,sex,no_people_in_household,highest_grade_completed,distance_fetch_water_one_way,type_of_fuel,water_source,toilet_type
0,1,60645a36a2237b3f54690be5,Mabwe,"0.5951310726,30.15312827",35.0,2.0,3.0,2.0,4.0,3.0,9.0,2.0
1,8,60645af8a2237b3f5469146c,Bihondo,"0.5785313656,30.1705658",45.0,1.0,7.0,0.0,5.0,1.0,9.0,2.0
2,9,60645921a2237b3f5468ffc1,Okabi,"3.125250327,30.84880735",64.0,2.0,4.0,0.0,4.0,7.0,1.0,2.0
3,15,60d06f3edefc3909beb58988,Ngoro,"0.1966,30.1683",28.0,1.0,4.0,4.0,6.0,1.0,1.0,2.0
4,22,60d06f50defc3909beb589a0,Kyigabo,"0.2642,30.1084",35.0,1.0,5.0,2.0,6.0,1.0,3.0,2.0
5,25,60d06f53defc3909beb589a9,Kazinga,"0.2642,30.1084",31.0,2.0,5.0,0.0,6.0,1.0,1.0,2.0
6,26,60d06f60defc3909beb589b1,Kitumba,"0.5897,30.2549",37.0,1.0,4.0,1.0,5.0,1.0,3.0,2.0
7,30,60d06f63defc3909beb589bc,Kakanju,"0.5424,30.1965",28.0,2.0,3.0,2.0,6.0,3.0,1.0,2.0
8,31,60d06f64defc3909beb589be,Kaburegye,"0.5424,30.1965",45.0,2.0,4.0,0.0,5.0,3.0,3.0,2.0
9,35,60d06f67defc3909beb589c6,Karambi,"0.1699,30.0781",26.0,2.0,3.0,2.0,5.0,3.0,1.0,3.0


**sex**   *'sex'* ----- 
1 = Male,
2 = Female

**How many people live in your household?**   *no_people_in_household* -----

**Highest grade completed in general education (# of years completed)**   *highest_grade_completed* -----                       
0 = Not completed/Never enrolled,
1 = Completed primary school,
2 = Completed secondary school,
3 = Completed tertiary school,
4 = Completed (Bachelor/Masters/PhD)

**Is this household connected to electricity supply?**   *'electrical_connection'* ----- 
1 = Yes,
2 = No

**How far does the household have to go (one way) to fetch water?**   *distance_fetch_water_one_way* -----
1 = Less than 20 metres,
2 = 20m - less than 50m,
3 = 50m - less than 100m,
4 = 100m - less than 500m,
5 = 500m - less than 1km,
6 = More than 1km

**How satisfied is the participant with their financial situation**   *financial_satisfaction* -----
1 = Very satisfied,
2 = Satisfied,
3 = Neither satisfied nor dissatisfied,
4 = Dissatisfied,
5 = Very dissatisfied

**What is the household's main source of energy for cooking?**     *cooking_energy_source* -----
1 = Wood
2 = Paraffin/Oil
3 = Coal/charcoal
4 = Gas
5 = Dung
6 = 1,2
7 = 1,3

**What is the source of water most often used in this household for drinking, bathing and washing clothes?**     *water_source* -----    
1 = Piped public tap/kiosk (free)
2 = Piped public tap/kiosk (paid for)
3 = Piped internal
4 = Piped tap outside
5 = Water carrier/tanker
6 = Well (non borehole)
7 = Borehole/ hand pump
8 = Rainwater tank
9 = Flowing river/stream
10 = Dam/stagnent water
11 = Protected spring
12 = Others
13 = 6,11
14 = 9,11
15 = 1,9
16 = 1,7
17 = 4,12
18 = 3,4
19 = 7,9
20 = 6,9,11
21 = 4,9


**What kind of toilet does the household use?**     *toilet_type* -----
1 = Flush toilet
2 = Pit latrine
3 = Community toilet
4 = Bucket toilet
5 = None/use fields etc.
6 = 1,2

**How often do you listen to the Radio?**     *radio_listening_frequency* -----
1 = Every day
2 = 2-3 times a week
3 = Once a week
4 = Occassionally
5 = Never

**How often do you watch TV?**     *tv_watching_frequency* -----
1 = Every day
2 = 2-3 times a week
3 = Once a week
4 = Occassionally
5 = Never

**How often do you read the newspaper?**     *newspaper_reading_frequency* -----
1 = Every day
2 = 2-3 times a week
3 = Once a week
4 = Occassionally
5 = Never

**How often do you use the internet?**     *internet_use_frequency* -----
1 = Every day
2 = 2-3 times a week
3 = Once a week
4 = Occassionally
5 = Never


In [22]:
df_result = pd.concat([df_only_geo, df_new_data_order], axis=1)
df_result
#We will base following analysis with this 'result' 

,index,Interview ID,Village,"lat,long",age,sex,no_people_in_household,highest_grade_completed,distance_fetch_water_one_way,type_of_fuel,...,Unburden,Comfort,Water Security,Medical treatment,Appeal (senses),Personal performance,Aspiration,Time management,Income,Reliability
0,1,60645a36a2237b3f54690be5,Mabwe,"0.5951310726,30.15312827",35.0,2.0,3.0,2.0,4.0,3.0,...,1,1,0,0,1,0,1,0,1,1
1,8,60645af8a2237b3f5469146c,Bihondo,"0.5785313656,30.1705658",45.0,1.0,7.0,0.0,5.0,1.0,...,0,1,1,1,1,1,1,0,1,0
2,9,60645921a2237b3f5468ffc1,Okabi,"3.125250327,30.84880735",64.0,2.0,4.0,0.0,4.0,7.0,...,0,1,0,1,0,1,1,0,1,0
3,15,60d06f3edefc3909beb58988,Ngoro,"0.1966,30.1683",28.0,1.0,4.0,4.0,6.0,1.0,...,1,0,1,1,0,1,0,0,1,0
4,22,60d06f50defc3909beb589a0,Kyigabo,"0.2642,30.1084",35.0,1.0,5.0,2.0,6.0,1.0,...,1,1,0,0,0,1,0,1,1,0
5,25,60d06f53defc3909beb589a9,Kazinga,"0.2642,30.1084",31.0,2.0,5.0,0.0,6.0,1.0,...,0,1,0,0,0,1,0,0,1,0
6,26,60d06f60defc3909beb589b1,Kitumba,"0.5897,30.2549",37.0,1.0,4.0,1.0,5.0,1.0,...,1,1,1,1,0,1,0,0,1,1
7,30,60d06f63defc3909beb589bc,Kakanju,"0.5424,30.1965",28.0,2.0,3.0,2.0,6.0,3.0,...,0,0,0,0,0,1,1,0,1,0
8,31,60d06f64defc3909beb589be,Kaburegye,"0.5424,30.1965",45.0,2.0,4.0,0.0,5.0,3.0,...,1,0,1,1,0,1,0,0,1,0
9,35,60d06f67defc3909beb589c6,Karambi,"0.1699,30.0781",26.0,2.0,3.0,2.0,5.0,3.0,...,1,1,1,1,0,1,0,0,1,1


# Preparation of Coordinates

In [23]:
df_only_geo_2 = df_only_geo['lat,long'].str.split(',', expand=True)
df_only_geo_3 = df_only_geo_2.rename(columns={0:"lat", 1: "long"})
del df_only_geo['lat,long']

a = list(df_only_geo_3['lat'])
b = list(df_only_geo_3['long'])

A = [float(x) for x in a]
#print(type(A[0]))
#print(A)

B = [float(x) for x in b]
#print(type(B[0]))
#print(B)

lst=list(zip(B,A))
#print(lst)

df_only_geo.insert(3, 'lat', A)
df_only_geo.insert(4, 'long', B)
df_only_geo.insert(5, 'long,lat', lst)
df_only_geo.head(15)

,index,Interview ID,Village,lat,long,"long,lat",age,sex,no_people_in_household,highest_grade_completed,distance_fetch_water_one_way,type_of_fuel,water_source,toilet_type
0,1,60645a36a2237b3f54690be5,Mabwe,0.595131,30.153128,"(30.15312827, 0.5951310726)",35.0,2.0,3.0,2.0,4.0,3.0,9.0,2.0
1,8,60645af8a2237b3f5469146c,Bihondo,0.578531,30.170566,"(30.1705658, 0.5785313656)",45.0,1.0,7.0,0.0,5.0,1.0,9.0,2.0
2,9,60645921a2237b3f5468ffc1,Okabi,3.125250,30.848807,"(30.84880735, 3.125250327)",64.0,2.0,4.0,0.0,4.0,7.0,1.0,2.0
3,15,60d06f3edefc3909beb58988,Ngoro,0.196600,30.168300,"(30.1683, 0.1966)",28.0,1.0,4.0,4.0,6.0,1.0,1.0,2.0
4,22,60d06f50defc3909beb589a0,Kyigabo,0.264200,30.108400,"(30.1084, 0.2642)",35.0,1.0,5.0,2.0,6.0,1.0,3.0,2.0
5,25,60d06f53defc3909beb589a9,Kazinga,0.264200,30.108400,"(30.1084, 0.2642)",31.0,2.0,5.0,0.0,6.0,1.0,1.0,2.0
6,26,60d06f60defc3909beb589b1,Kitumba,0.589700,30.254900,"(30.2549, 0.5897)",37.0,1.0,4.0,1.0,5.0,1.0,3.0,2.0
7,30,60d06f63defc3909beb589bc,Kakanju,0.542400,30.196500,"(30.1965, 0.5424)",28.0,2.0,3.0,2.0,6.0,3.0,1.0,2.0
8,31,60d06f64defc3909beb589be,Kaburegye,0.542400,30.196500,"(30.1965, 0.5424)",45.0,2.0,4.0,0.0,5.0,3.0,3.0,2.0
9,35,60d06f67defc3909beb589c6,Karambi,0.169900,30.078100,"(30.0781, 0.1699)",26.0,2.0,3.0,2.0,5.0,3.0,1.0,3.0


In [24]:
print(type(df_only_geo.loc[3,'lat']))
print(type(df_only_geo.loc[3,'long']))
print(type(df_only_geo.loc[3,'long,lat']))

<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'tuple'>


# Precipitation Analysis

In [25]:
# Filepaths to raster
precipitation_raster = "Precipitation_Uganda_2016.tif"
#https://giovanni.gsfc.nasa.gov/giovanni/

In [26]:
# Instantiate list to hold precipitation info
precipitation = list(range(0,len(df_only_geo)))

# Read in precipitation data

precipitation_gdata = gdal.Open(precipitation_raster)
precipitation_gt = precipitation_gdata.GetGeoTransform()
precipitation_data = precipitation_gdata.ReadAsArray().astype(np.float)

# Fake list of cluster_ids
cluster_ids = list(range(0,len(df_only_geo)))

# Fake list of mean coordinates
mean_coord = list(df_only_geo['long,lat'])
#print(mean_coord)
#print(type(mean_coord))

#mean_coord = {
#    0: (32.92603986, 3.668252174),
#    1: (32.8443, -0.7959),
#    2: (29.7351, -1.1585),
#    3: (32.92603986, 3.668252174),
#    4: (32.92603986, 3.668252174),
#}

# Get information about each cluster. For reference: in the geotransform loaded from GDAL,
# [0] = top left x, [1] = w-e pixel resolution, [2] = 0, [3] = top left y, [4] = 0, [5] = n-s pixel resolution
#(29.499999237060546, 0.09999999956750111, 0.0, 4.299999999588933, 0.0, -0.09999999956750111)

for cluster in cluster_ids:
    # Get the wind value at the mean point of the cluster
    x = int((mean_coord[cluster][0] - precipitation_gt[0])/precipitation_gt[1])
    #print(x)
    y = int((mean_coord[cluster][1] - precipitation_gt[3])/precipitation_gt[5])
    #print(y)
    precipitation[cluster] = precipitation_data[y, x]
    
df_only_geo.insert(14, 'Precipitation (mm)', precipitation)
print(type(df_only_geo.loc[3,'Precipitation (mm)']))
df_only_geo

#Close all dataset
#wind_gdata = None

<class 'numpy.float64'>


,index,Interview ID,Village,lat,long,"long,lat",age,sex,no_people_in_household,highest_grade_completed,distance_fetch_water_one_way,type_of_fuel,water_source,toilet_type,Precipitation (mm)
0,1,60645a36a2237b3f54690be5,Mabwe,0.595131,30.153128,"(30.15312827, 0.5951310726)",35.0,2.0,3.0,2.0,4.0,3.0,9.0,2.0,1482.106567
1,8,60645af8a2237b3f5469146c,Bihondo,0.578531,30.170566,"(30.1705658, 0.5785313656)",45.0,1.0,7.0,0.0,5.0,1.0,9.0,2.0,1482.106567
2,9,60645921a2237b3f5468ffc1,Okabi,3.125250,30.848807,"(30.84880735, 3.125250327)",64.0,2.0,4.0,0.0,4.0,7.0,1.0,2.0,1345.285400
3,15,60d06f3edefc3909beb58988,Ngoro,0.196600,30.168300,"(30.1683, 0.1966)",28.0,1.0,4.0,4.0,6.0,1.0,1.0,2.0,891.146545
4,22,60d06f50defc3909beb589a0,Kyigabo,0.264200,30.108400,"(30.1084, 0.2642)",35.0,1.0,5.0,2.0,6.0,1.0,3.0,2.0,977.963806
5,25,60d06f53defc3909beb589a9,Kazinga,0.264200,30.108400,"(30.1084, 0.2642)",31.0,2.0,5.0,0.0,6.0,1.0,1.0,2.0,977.963806
6,26,60d06f60defc3909beb589b1,Kitumba,0.589700,30.254900,"(30.2549, 0.5897)",37.0,1.0,4.0,1.0,5.0,1.0,3.0,2.0,1247.092651
7,30,60d06f63defc3909beb589bc,Kakanju,0.542400,30.196500,"(30.1965, 0.5424)",28.0,2.0,3.0,2.0,6.0,3.0,1.0,2.0,1482.106567
8,31,60d06f64defc3909beb589be,Kaburegye,0.542400,30.196500,"(30.1965, 0.5424)",45.0,2.0,4.0,0.0,5.0,3.0,3.0,2.0,1482.106567
9,35,60d06f67defc3909beb589c6,Karambi,0.169900,30.078100,"(30.0781, 0.1699)",26.0,2.0,3.0,2.0,5.0,3.0,1.0,3.0,1003.117432


# Travel Time Analysis

In [27]:
# Filepaths to raster
#travel_time_raster = "Travel_Time_malariaatlas_2015_2.tif"
travel_time_raster = "travel_time.tif"

In [28]:
# Instantiate list to hold travel time info
travel_time = list(range(0,len(df_only_geo)))

# Read in travel time data

travel_time_gdata = gdal.Open(travel_time_raster)
travel_time_gt = travel_time_gdata.GetGeoTransform()
travel_time_data = travel_time_gdata.ReadAsArray().astype(np.float)

# Fake list of cluster_ids
cluster_ids = list(range(0,len(df_only_geo)))

# Fake list of mean coordinates
mean_coord = list(df_only_geo['long,lat'])
#print(mean_coord)
#print(type(mean_coord))

# Get information about each cluster. For reference: in the geotransform loaded from GDAL,
# [0] = top left x, [1] = w-e pixel resolution, [2] = 0, [3] = top left y, [4] = 0, [5] = n-s pixel resolution

for cluster in cluster_ids:
    # Get the wind value at the mean point of the cluster
    x = int((mean_coord[cluster][0] - travel_time_gt[0])/travel_time_gt[1])
    #print(x)
    y = int((mean_coord[cluster][1] - travel_time_gt[3])/travel_time_gt[5])
    #print(y)
    travel_time[cluster] = travel_time_data[y, x]

df_only_geo.insert(14, 'Travel Time (min)', travel_time)

#print(type(df_only_geo.loc[3,'Travel Time (min)']))
df_only_geo

,index,Interview ID,Village,lat,long,"long,lat",age,sex,no_people_in_household,highest_grade_completed,distance_fetch_water_one_way,type_of_fuel,water_source,toilet_type,Travel Time (min),Precipitation (mm)
0,1,60645a36a2237b3f54690be5,Mabwe,0.595131,30.153128,"(30.15312827, 0.5951310726)",35.0,2.0,3.0,2.0,4.0,3.0,9.0,2.0,24.0,1482.106567
1,8,60645af8a2237b3f5469146c,Bihondo,0.578531,30.170566,"(30.1705658, 0.5785313656)",45.0,1.0,7.0,0.0,5.0,1.0,9.0,2.0,19.0,1482.106567
2,9,60645921a2237b3f5468ffc1,Okabi,3.125250,30.848807,"(30.84880735, 3.125250327)",64.0,2.0,4.0,0.0,4.0,7.0,1.0,2.0,21.0,1345.285400
3,15,60d06f3edefc3909beb58988,Ngoro,0.196600,30.168300,"(30.1683, 0.1966)",28.0,1.0,4.0,4.0,6.0,1.0,1.0,2.0,28.0,891.146545
4,22,60d06f50defc3909beb589a0,Kyigabo,0.264200,30.108400,"(30.1084, 0.2642)",35.0,1.0,5.0,2.0,6.0,1.0,3.0,2.0,13.0,977.963806
5,25,60d06f53defc3909beb589a9,Kazinga,0.264200,30.108400,"(30.1084, 0.2642)",31.0,2.0,5.0,0.0,6.0,1.0,1.0,2.0,13.0,977.963806
6,26,60d06f60defc3909beb589b1,Kitumba,0.589700,30.254900,"(30.2549, 0.5897)",37.0,1.0,4.0,1.0,5.0,1.0,3.0,2.0,10.0,1247.092651
7,30,60d06f63defc3909beb589bc,Kakanju,0.542400,30.196500,"(30.1965, 0.5424)",28.0,2.0,3.0,2.0,6.0,3.0,1.0,2.0,21.0,1482.106567
8,31,60d06f64defc3909beb589be,Kaburegye,0.542400,30.196500,"(30.1965, 0.5424)",45.0,2.0,4.0,0.0,5.0,3.0,3.0,2.0,21.0,1482.106567
9,35,60d06f67defc3909beb589c6,Karambi,0.169900,30.078100,"(30.0781, 0.1699)",26.0,2.0,3.0,2.0,5.0,3.0,1.0,3.0,0.0,1003.117432


# Luminance Analysis (Night Lights)

In [29]:
# Filepaths to raster BAND1
luminance1_raster = "luminance_uganda_band1.tif"
#https://earthobservatory.nasa.gov/features/NightLights - Download Full Resolution (500m) By Region - 2016 Color (C1 and C2)

In [30]:
luminance1 = list(range(0,len(df_only_geo)))

# Read in travel time data

luminance1_gdata = gdal.Open(luminance1_raster)
luminance1_gt = luminance1_gdata.GetGeoTransform()
luminance1_data = luminance1_gdata.ReadAsArray().astype(np.float)
# Fake list of cluster_ids
cluster_ids = list(range(0,len(df_only_geo)))

# Fake list of mean coordinates
mean_coord = list(df_only_geo['long,lat'])
#print(mean_coord)
#print(type(mean_coord))

# Get information about each cluster. For reference: in the geotransform loaded from GDAL,
# [0] = top left x, [1] = w-e pixel resolution, [2] = 0, [3] = top left y, [4] = 0, [5] = n-s pixel resolution

#(29.220833333,  0.0041666666666666675,  0.0,  4.533333333,  0.0,  -0.004166666666442501)

for cluster in cluster_ids:
    # Get the wind value at the mean point of the cluster
    x = int((mean_coord[cluster][0] - luminance1_gt[0])/luminance1_gt[1])
    #print(x)
    y = int((mean_coord[cluster][1] - luminance1_gt[3])/luminance1_gt[5])
    #print(y)
    luminance1[cluster] = luminance1_data[y, x]

df_only_geo.insert(14, 'Luminance Band 1', luminance1)

df_only_geo

,index,Interview ID,Village,lat,long,"long,lat",age,sex,no_people_in_household,highest_grade_completed,distance_fetch_water_one_way,type_of_fuel,water_source,toilet_type,Luminance Band 1,Travel Time (min),Precipitation (mm)
0,1,60645a36a2237b3f54690be5,Mabwe,0.595131,30.153128,"(30.15312827, 0.5951310726)",35.0,2.0,3.0,2.0,4.0,3.0,9.0,2.0,15.0,24.0,1482.106567
1,8,60645af8a2237b3f5469146c,Bihondo,0.578531,30.170566,"(30.1705658, 0.5785313656)",45.0,1.0,7.0,0.0,5.0,1.0,9.0,2.0,14.0,19.0,1482.106567
2,9,60645921a2237b3f5468ffc1,Okabi,3.125250,30.848807,"(30.84880735, 3.125250327)",64.0,2.0,4.0,0.0,4.0,7.0,1.0,2.0,16.0,21.0,1345.285400
3,15,60d06f3edefc3909beb58988,Ngoro,0.196600,30.168300,"(30.1683, 0.1966)",28.0,1.0,4.0,4.0,6.0,1.0,1.0,2.0,12.0,28.0,891.146545
4,22,60d06f50defc3909beb589a0,Kyigabo,0.264200,30.108400,"(30.1084, 0.2642)",35.0,1.0,5.0,2.0,6.0,1.0,3.0,2.0,15.0,13.0,977.963806
5,25,60d06f53defc3909beb589a9,Kazinga,0.264200,30.108400,"(30.1084, 0.2642)",31.0,2.0,5.0,0.0,6.0,1.0,1.0,2.0,15.0,13.0,977.963806
6,26,60d06f60defc3909beb589b1,Kitumba,0.589700,30.254900,"(30.2549, 0.5897)",37.0,1.0,4.0,1.0,5.0,1.0,3.0,2.0,12.0,10.0,1247.092651
7,30,60d06f63defc3909beb589bc,Kakanju,0.542400,30.196500,"(30.1965, 0.5424)",28.0,2.0,3.0,2.0,6.0,3.0,1.0,2.0,13.0,21.0,1482.106567
8,31,60d06f64defc3909beb589be,Kaburegye,0.542400,30.196500,"(30.1965, 0.5424)",45.0,2.0,4.0,0.0,5.0,3.0,3.0,2.0,13.0,21.0,1482.106567
9,35,60d06f67defc3909beb589c6,Karambi,0.169900,30.078100,"(30.0781, 0.1699)",26.0,2.0,3.0,2.0,5.0,3.0,1.0,3.0,20.0,0.0,1003.117432


In [31]:
# Filepaths to raster BAND2
luminance2_raster = "luminance_uganda_band2.tif"

In [32]:
luminance2 = list(range(0,len(df_only_geo)))

# Read in travel time data

luminance2_gdata = gdal.Open(luminance2_raster)
luminance2_gt = luminance2_gdata.GetGeoTransform()
luminance2_data = luminance2_gdata.ReadAsArray().astype(np.float)

# Fake list of cluster_ids
cluster_ids = list(range(0,len(df_only_geo)))

# Fake list of mean coordinates
mean_coord = list(df_only_geo['long,lat'])
#print(mean_coord)
#print(type(mean_coord))

# Get information about each cluster. For reference: in the geotransform loaded from GDAL,
# [0] = top left x, [1] = w-e pixel resolution, [2] = 0, [3] = top left y, [4] = 0, [5] = n-s pixel resolution

#(29.220833333,  0.0041666666666666675,  0.0,  4.533333333,  0.0,  -0.004166666666442501)

for cluster in cluster_ids:
    # Get the wind value at the mean point of the cluster
    x = int((mean_coord[cluster][0] - luminance2_gt[0])/luminance2_gt[1])
    #print(x)
    y = int((mean_coord[cluster][1] - luminance2_gt[3])/luminance2_gt[5])
    #print(y)
    luminance2[cluster] = luminance2_data[y, x]

df_only_geo.insert(14, 'Luminance Band 2', luminance2)

df_only_geo

,index,Interview ID,Village,lat,long,"long,lat",age,sex,no_people_in_household,highest_grade_completed,distance_fetch_water_one_way,type_of_fuel,water_source,toilet_type,Luminance Band 2,Luminance Band 1,Travel Time (min),Precipitation (mm)
0,1,60645a36a2237b3f54690be5,Mabwe,0.595131,30.153128,"(30.15312827, 0.5951310726)",35.0,2.0,3.0,2.0,4.0,3.0,9.0,2.0,17.0,15.0,24.0,1482.106567
1,8,60645af8a2237b3f5469146c,Bihondo,0.578531,30.170566,"(30.1705658, 0.5785313656)",45.0,1.0,7.0,0.0,5.0,1.0,9.0,2.0,17.0,14.0,19.0,1482.106567
2,9,60645921a2237b3f5468ffc1,Okabi,3.125250,30.848807,"(30.84880735, 3.125250327)",64.0,2.0,4.0,0.0,4.0,7.0,1.0,2.0,18.0,16.0,21.0,1345.285400
3,15,60d06f3edefc3909beb58988,Ngoro,0.196600,30.168300,"(30.1683, 0.1966)",28.0,1.0,4.0,4.0,6.0,1.0,1.0,2.0,15.0,12.0,28.0,891.146545
4,22,60d06f50defc3909beb589a0,Kyigabo,0.264200,30.108400,"(30.1084, 0.2642)",35.0,1.0,5.0,2.0,6.0,1.0,3.0,2.0,17.0,15.0,13.0,977.963806
5,25,60d06f53defc3909beb589a9,Kazinga,0.264200,30.108400,"(30.1084, 0.2642)",31.0,2.0,5.0,0.0,6.0,1.0,1.0,2.0,17.0,15.0,13.0,977.963806
6,26,60d06f60defc3909beb589b1,Kitumba,0.589700,30.254900,"(30.2549, 0.5897)",37.0,1.0,4.0,1.0,5.0,1.0,3.0,2.0,16.0,12.0,10.0,1247.092651
7,30,60d06f63defc3909beb589bc,Kakanju,0.542400,30.196500,"(30.1965, 0.5424)",28.0,2.0,3.0,2.0,6.0,3.0,1.0,2.0,16.0,13.0,21.0,1482.106567
8,31,60d06f64defc3909beb589be,Kaburegye,0.542400,30.196500,"(30.1965, 0.5424)",45.0,2.0,4.0,0.0,5.0,3.0,3.0,2.0,16.0,13.0,21.0,1482.106567
9,35,60d06f67defc3909beb589c6,Karambi,0.169900,30.078100,"(30.0781, 0.1699)",26.0,2.0,3.0,2.0,5.0,3.0,1.0,3.0,21.0,20.0,0.0,1003.117432


In [33]:
# Filepaths to raster BAND3
luminance3_raster = "luminance_uganda_band3.tif"

In [34]:
luminance3 = list(range(0,len(df_only_geo)))

# Read in travel time data

luminance3_gdata = gdal.Open(luminance3_raster)
luminance3_gt = luminance3_gdata.GetGeoTransform()
luminance3_data = luminance3_gdata.ReadAsArray().astype(np.float)
# Fake list of cluster_ids
cluster_ids = list(range(0,len(df_only_geo)))

# Fake list of mean coordinates
mean_coord = list(df_only_geo['long,lat'])
#print(mean_coord)
#print(type(mean_coord))

# Get information about each cluster. For reference: in the geotransform loaded from GDAL,
# [0] = top left x, [1] = w-e pixel resolution, [2] = 0, [3] = top left y, [4] = 0, [5] = n-s pixel resolution

#(29.220833333,  0.0041666666666666675,  0.0,  4.533333333,  0.0,  -0.004166666666442501)

for cluster in cluster_ids:
    # Get the wind value at the mean point of the cluster
    x = int((mean_coord[cluster][0] - luminance3_gt[0])/luminance3_gt[1])
    #print(x)
    y = int((mean_coord[cluster][1] - luminance3_gt[3])/luminance3_gt[5])
    #print(y)
    luminance3[cluster] = luminance3_data[y, x]

df_only_geo.insert(14, 'Luminance Band 3', luminance3)
df_only_geo

,index,Interview ID,Village,lat,long,"long,lat",age,sex,no_people_in_household,highest_grade_completed,distance_fetch_water_one_way,type_of_fuel,water_source,toilet_type,Luminance Band 3,Luminance Band 2,Luminance Band 1,Travel Time (min),Precipitation (mm)
0,1,60645a36a2237b3f54690be5,Mabwe,0.595131,30.153128,"(30.15312827, 0.5951310726)",35.0,2.0,3.0,2.0,4.0,3.0,9.0,2.0,29.0,17.0,15.0,24.0,1482.106567
1,8,60645af8a2237b3f5469146c,Bihondo,0.578531,30.170566,"(30.1705658, 0.5785313656)",45.0,1.0,7.0,0.0,5.0,1.0,9.0,2.0,29.0,17.0,14.0,19.0,1482.106567
2,9,60645921a2237b3f5468ffc1,Okabi,3.125250,30.848807,"(30.84880735, 3.125250327)",64.0,2.0,4.0,0.0,4.0,7.0,1.0,2.0,32.0,18.0,16.0,21.0,1345.285400
3,15,60d06f3edefc3909beb58988,Ngoro,0.196600,30.168300,"(30.1683, 0.1966)",28.0,1.0,4.0,4.0,6.0,1.0,1.0,2.0,27.0,15.0,12.0,28.0,891.146545
4,22,60d06f50defc3909beb589a0,Kyigabo,0.264200,30.108400,"(30.1084, 0.2642)",35.0,1.0,5.0,2.0,6.0,1.0,3.0,2.0,32.0,17.0,15.0,13.0,977.963806
5,25,60d06f53defc3909beb589a9,Kazinga,0.264200,30.108400,"(30.1084, 0.2642)",31.0,2.0,5.0,0.0,6.0,1.0,1.0,2.0,32.0,17.0,15.0,13.0,977.963806
6,26,60d06f60defc3909beb589b1,Kitumba,0.589700,30.254900,"(30.2549, 0.5897)",37.0,1.0,4.0,1.0,5.0,1.0,3.0,2.0,26.0,16.0,12.0,10.0,1247.092651
7,30,60d06f63defc3909beb589bc,Kakanju,0.542400,30.196500,"(30.1965, 0.5424)",28.0,2.0,3.0,2.0,6.0,3.0,1.0,2.0,27.0,16.0,13.0,21.0,1482.106567
8,31,60d06f64defc3909beb589be,Kaburegye,0.542400,30.196500,"(30.1965, 0.5424)",45.0,2.0,4.0,0.0,5.0,3.0,3.0,2.0,27.0,16.0,13.0,21.0,1482.106567
9,35,60d06f67defc3909beb589c6,Karambi,0.169900,30.078100,"(30.0781, 0.1699)",26.0,2.0,3.0,2.0,5.0,3.0,1.0,3.0,41.0,21.0,20.0,0.0,1003.117432


In [35]:
#Relative luminance formula E'Y = 0.2126E'R + 0.7152E'G + 0.0722E'B
#E'R is the red channel, E'G is the green channel, and E'B is the blue channel
lum1 = np.array(luminance1)
lum2 = np.array(luminance2)
lum3 = np.array(luminance3)
luminance = 0.2126 * lum1 + 0.7152 * lum2 + 0.0722 * lum3
luminance = luminance.tolist()

df_only_geo.insert(14, 'Luminance', luminance)
df_only_geo

,index,Interview ID,Village,lat,long,"long,lat",age,sex,no_people_in_household,highest_grade_completed,distance_fetch_water_one_way,type_of_fuel,water_source,toilet_type,Luminance,Luminance Band 3,Luminance Band 2,Luminance Band 1,Travel Time (min),Precipitation (mm)
0,1,60645a36a2237b3f54690be5,Mabwe,0.595131,30.153128,"(30.15312827, 0.5951310726)",35.0,2.0,3.0,2.0,4.0,3.0,9.0,2.0,17.4412,29.0,17.0,15.0,24.0,1482.106567
1,8,60645af8a2237b3f5469146c,Bihondo,0.578531,30.170566,"(30.1705658, 0.5785313656)",45.0,1.0,7.0,0.0,5.0,1.0,9.0,2.0,17.2286,29.0,17.0,14.0,19.0,1482.106567
2,9,60645921a2237b3f5468ffc1,Okabi,3.125250,30.848807,"(30.84880735, 3.125250327)",64.0,2.0,4.0,0.0,4.0,7.0,1.0,2.0,18.5856,32.0,18.0,16.0,21.0,1345.285400
3,15,60d06f3edefc3909beb58988,Ngoro,0.196600,30.168300,"(30.1683, 0.1966)",28.0,1.0,4.0,4.0,6.0,1.0,1.0,2.0,15.2286,27.0,15.0,12.0,28.0,891.146545
4,22,60d06f50defc3909beb589a0,Kyigabo,0.264200,30.108400,"(30.1084, 0.2642)",35.0,1.0,5.0,2.0,6.0,1.0,3.0,2.0,17.6578,32.0,17.0,15.0,13.0,977.963806
5,25,60d06f53defc3909beb589a9,Kazinga,0.264200,30.108400,"(30.1084, 0.2642)",31.0,2.0,5.0,0.0,6.0,1.0,1.0,2.0,17.6578,32.0,17.0,15.0,13.0,977.963806
6,26,60d06f60defc3909beb589b1,Kitumba,0.589700,30.254900,"(30.2549, 0.5897)",37.0,1.0,4.0,1.0,5.0,1.0,3.0,2.0,15.8716,26.0,16.0,12.0,10.0,1247.092651
7,30,60d06f63defc3909beb589bc,Kakanju,0.542400,30.196500,"(30.1965, 0.5424)",28.0,2.0,3.0,2.0,6.0,3.0,1.0,2.0,16.1564,27.0,16.0,13.0,21.0,1482.106567
8,31,60d06f64defc3909beb589be,Kaburegye,0.542400,30.196500,"(30.1965, 0.5424)",45.0,2.0,4.0,0.0,5.0,3.0,3.0,2.0,16.1564,27.0,16.0,13.0,21.0,1482.106567
9,35,60d06f67defc3909beb589c6,Karambi,0.169900,30.078100,"(30.0781, 0.1699)",26.0,2.0,3.0,2.0,5.0,3.0,1.0,3.0,22.2314,41.0,21.0,20.0,0.0,1003.117432


# Land Cover Analysis

#####  0 unknown, 111 ENF_closed, 112 EBF_closed, 113 DNF_closed, 114 DBF_closed, 115 mixed _closed, 116 unknown_closed, 121 ENF_open, 122 EBF_open, 123 DNF_open, 124 DBF_open, 125 mixed _open, 126 unknown_open, 20 shrubland, 30 herbaceous_vegetation, 40 cropland, 50 built-up, 60 bare_sparse_vegetation, 70 snow_ice, 80 permanent_inland_water, 90 herbaceous_wetland, 100 moss_lichen, 200 sea

##### https://lcviewer.vito.be/2016/Uganda
##### https://lcviewer.vito.be/download

In [36]:
# Filepaths to raster BAND3
land_cover_raster = "land_cover_uganda.tif"

In [37]:
land_cover = list(range(0,len(df_only_geo)))

# Read in travel time data

land_cover_gdata = gdal.Open(land_cover_raster)
land_cover_gt = land_cover_gdata.GetGeoTransform()
land_cover_data = land_cover_gdata.ReadAsArray().astype(np.float)

# Fake list of cluster_ids
cluster_ids = list(range(0,len(df_only_geo)))

# Fake list of mean coordinates
mean_coord = list(df_only_geo['long,lat'])
#print(mean_coord)
#print(type(mean_coord))

# Get information about each cluster. For reference: in the geotransform loaded from GDAL,
# [0] = top left x, [1] = w-e pixel resolution, [2] = 0, [3] = top left y, [4] = 0, [5] = n-s pixel resolution

#(29.220833333,  0.0041666666666666675,  0.0,  4.533333333,  0.0,  -0.004166666666442501)

for cluster in cluster_ids:
    # Get the wind value at the mean point of the cluster
    x = int((mean_coord[cluster][0] - land_cover_gt[0])/land_cover_gt[1])
    #print(x)
    y = int((mean_coord[cluster][1] - land_cover_gt[3])/land_cover_gt[5])
    #print(y)
    land_cover[cluster] = land_cover_data[y, x]

df_only_geo.insert(14, 'Land Cover', land_cover)
df_only_geo

,index,Interview ID,Village,lat,long,"long,lat",age,sex,no_people_in_household,highest_grade_completed,...,type_of_fuel,water_source,toilet_type,Land Cover,Luminance,Luminance Band 3,Luminance Band 2,Luminance Band 1,Travel Time (min),Precipitation (mm)
0,1,60645a36a2237b3f54690be5,Mabwe,0.595131,30.153128,"(30.15312827, 0.5951310726)",35.0,2.0,3.0,2.0,...,3.0,9.0,2.0,40.0,17.4412,29.0,17.0,15.0,24.0,1482.106567
1,8,60645af8a2237b3f5469146c,Bihondo,0.578531,30.170566,"(30.1705658, 0.5785313656)",45.0,1.0,7.0,0.0,...,1.0,9.0,2.0,40.0,17.2286,29.0,17.0,14.0,19.0,1482.106567
2,9,60645921a2237b3f5468ffc1,Okabi,3.125250,30.848807,"(30.84880735, 3.125250327)",64.0,2.0,4.0,0.0,...,7.0,1.0,2.0,40.0,18.5856,32.0,18.0,16.0,21.0,1345.285400
3,15,60d06f3edefc3909beb58988,Ngoro,0.196600,30.168300,"(30.1683, 0.1966)",28.0,1.0,4.0,4.0,...,1.0,1.0,2.0,126.0,15.2286,27.0,15.0,12.0,28.0,891.146545
4,22,60d06f50defc3909beb589a0,Kyigabo,0.264200,30.108400,"(30.1084, 0.2642)",35.0,1.0,5.0,2.0,...,1.0,3.0,2.0,40.0,17.6578,32.0,17.0,15.0,13.0,977.963806
5,25,60d06f53defc3909beb589a9,Kazinga,0.264200,30.108400,"(30.1084, 0.2642)",31.0,2.0,5.0,0.0,...,1.0,1.0,2.0,40.0,17.6578,32.0,17.0,15.0,13.0,977.963806
6,26,60d06f60defc3909beb589b1,Kitumba,0.589700,30.254900,"(30.2549, 0.5897)",37.0,1.0,4.0,1.0,...,1.0,3.0,2.0,20.0,15.8716,26.0,16.0,12.0,10.0,1247.092651
7,30,60d06f63defc3909beb589bc,Kakanju,0.542400,30.196500,"(30.1965, 0.5424)",28.0,2.0,3.0,2.0,...,3.0,1.0,2.0,30.0,16.1564,27.0,16.0,13.0,21.0,1482.106567
8,31,60d06f64defc3909beb589be,Kaburegye,0.542400,30.196500,"(30.1965, 0.5424)",45.0,2.0,4.0,0.0,...,3.0,3.0,2.0,30.0,16.1564,27.0,16.0,13.0,21.0,1482.106567
9,35,60d06f67defc3909beb589c6,Karambi,0.169900,30.078100,"(30.0781, 0.1699)",26.0,2.0,3.0,2.0,...,3.0,1.0,3.0,50.0,22.2314,41.0,21.0,20.0,0.0,1003.117432


# DHS Data Preparation (Coordinates)

In [38]:
#Read DHS (Geographic Datasets) data and extract relevant parameters
DHS_geo = "DHS_Individual_2016.shp"
df_dhs_predata_geo = gpd.read_file(DHS_geo)
df_dhs_data_geo = df_dhs_predata_geo[['DHSCLUST', 'ADM1NAME', 'DHSREGNA', 'URBAN_RURA', 'LATNUM', 'LONGNUM', 'ALT_DEM', 'geometry']].copy()
df_dhs_data_geo

,DHSCLUST,ADM1NAME,DHSREGNA,URBAN_RURA,LATNUM,LONGNUM,ALT_DEM,geometry
0,1.0,KAMPALA,Kampala,U,0.320188,32.568206,1197.0,POINT (32.56821 0.32019)
1,2.0,KAMPALA,Kampala,U,0.340653,32.593627,1179.0,POINT (32.59363 0.34065)
2,3.0,KAMPALA,Kampala,U,0.313103,32.566556,1189.0,POINT (32.56656 0.31310)
3,4.0,KAMPALA,Kampala,U,0.353368,32.558144,1181.0,POINT (32.55814 0.35337)
4,5.0,KAMPALA,Kampala,U,0.367388,32.594357,1226.0,POINT (32.59436 0.36739)
...,...,...,...,...,...,...,...,...
691,693.0,MAYUGE,Busoga,R,0.242547,33.510958,1174.0,POINT (33.51096 0.24255)
692,694.0,MAYUGE,Busoga,R,0.223461,33.494763,1167.0,POINT (33.49476 0.22346)
693,695.0,MAYUGE,Busoga,R,0.266963,33.437440,1134.0,POINT (33.43744 0.26696)
694,696.0,MAYUGE,Busoga,R,0.451594,33.295100,1170.0,POINT (33.29510 0.45159)


In [39]:
#List of projected coordinates for communities in Stephi's data set (df_only_geo)

lat_comm = list(df_only_geo['lat'])
long_comm = list(df_only_geo['long'])

long_comm_c = list(range(0,len(df_only_geo)))
lat_comm_c = list(range(0,len(df_only_geo)))

i=0

proj_wgs84 = pyproj.Proj(init="epsg:4326")
proj_gk4 = pyproj.Proj(init="epsg:3857")

for (long_comm,lat_comm) in zip(long_comm,lat_comm):
    long_comm_c[i], lat_comm_c[i] = pyproj.transform(proj_wgs84, proj_gk4, long_comm, lat_comm)
    i=i+1

lat_clus = list(df_dhs_data_geo['LATNUM'])
long_clus = list(df_dhs_data_geo['LONGNUM'])

long_clus_c = list(range(0,len(df_dhs_data_geo)))
lat_clus_c = list(range(0,len(df_dhs_data_geo)))

i=0

proj_wgs84 = pyproj.Proj(init="epsg:4326")
proj_gk4 = pyproj.Proj(init="epsg:3857")

for (long_clus,lat_clus) in zip(long_clus,lat_clus):
    long_clus_c[i], lat_clus_c[i] = pyproj.transform(proj_wgs84, proj_gk4, long_clus, lat_clus)
    i=i+1
    
#display(long_clus_c)
#display(lat_clus_c)

# Proximity to National Roads Analysis

In [40]:
test_data = "Surface_Cl_Bituminous.shp"

# Instantiate list to hold travel time info
closest_national_road = list(range(0,len(df_only_geo)))

cluster_ids = list(range(0,len(df_only_geo)))
radius = 25000000 # metres
roads = gpd.read_file(test_data)
#print(roads)

for cluster in cluster_ids:
    point = Point(long_comm_c[cluster], lat_comm_c[cluster])
    buffer = point.buffer(radius)
    clipped_roads = gpd.clip(roads, buffer)
    
    # calculate distance for all lines
    distances = clipped_roads["geometry"].map(lambda line: point.distance(line))
    clipped_roads = clipped_roads.assign(Distance=distances)

    # get 4 roads closest to the point
    clipped_roads = clipped_roads.sort_values(by="Distance")
    closest_roads = clipped_roads.head(4)   
    closest_national_road[cluster]=closest_roads.loc[closest_roads.index.values[0], 'Distance']/1000
    
closest_national_road  
df_only_geo.insert(14, 'Closest National Road (km)', closest_national_road)
df_only_geo

,index,Interview ID,Village,lat,long,"long,lat",age,sex,no_people_in_household,highest_grade_completed,...,water_source,toilet_type,Closest National Road (km),Land Cover,Luminance,Luminance Band 3,Luminance Band 2,Luminance Band 1,Travel Time (min),Precipitation (mm)
0,1,60645a36a2237b3f54690be5,Mabwe,0.595131,30.153128,"(30.15312827, 0.5951310726)",35.0,2.0,3.0,2.0,...,9.0,2.0,4.495595,40.0,17.4412,29.0,17.0,15.0,24.0,1482.106567
1,8,60645af8a2237b3f5469146c,Bihondo,0.578531,30.170566,"(30.1705658, 0.5785313656)",45.0,1.0,7.0,0.0,...,9.0,2.0,1.842590,40.0,17.2286,29.0,17.0,14.0,19.0,1482.106567
2,9,60645921a2237b3f5468ffc1,Okabi,3.125250,30.848807,"(30.84880735, 3.125250327)",64.0,2.0,4.0,0.0,...,1.0,2.0,5.073637,40.0,18.5856,32.0,18.0,16.0,21.0,1345.285400
3,15,60d06f3edefc3909beb58988,Ngoro,0.196600,30.168300,"(30.1683, 0.1966)",28.0,1.0,4.0,4.0,...,1.0,2.0,6.338110,126.0,15.2286,27.0,15.0,12.0,28.0,891.146545
4,22,60d06f50defc3909beb589a0,Kyigabo,0.264200,30.108400,"(30.1084, 0.2642)",35.0,1.0,5.0,2.0,...,3.0,2.0,1.180557,40.0,17.6578,32.0,17.0,15.0,13.0,977.963806
5,25,60d06f53defc3909beb589a9,Kazinga,0.264200,30.108400,"(30.1084, 0.2642)",31.0,2.0,5.0,0.0,...,1.0,2.0,1.180557,40.0,17.6578,32.0,17.0,15.0,13.0,977.963806
6,26,60d06f60defc3909beb589b1,Kitumba,0.589700,30.254900,"(30.2549, 0.5897)",37.0,1.0,4.0,1.0,...,3.0,2.0,3.752815,20.0,15.8716,26.0,16.0,12.0,10.0,1247.092651
7,30,60d06f63defc3909beb589bc,Kakanju,0.542400,30.196500,"(30.1965, 0.5424)",28.0,2.0,3.0,2.0,...,1.0,2.0,2.745248,30.0,16.1564,27.0,16.0,13.0,21.0,1482.106567
8,31,60d06f64defc3909beb589be,Kaburegye,0.542400,30.196500,"(30.1965, 0.5424)",45.0,2.0,4.0,0.0,...,3.0,2.0,2.745248,30.0,16.1564,27.0,16.0,13.0,21.0,1482.106567
9,35,60d06f67defc3909beb589c6,Karambi,0.169900,30.078100,"(30.0781, 0.1699)",26.0,2.0,3.0,2.0,...,1.0,3.0,0.530190,50.0,22.2314,41.0,21.0,20.0,0.0,1003.117432


# Proximity to Roads Analysis

In [41]:
test_data = "Road_Uganda_2017_National(BitUns).shp"

# Instantiate list to hold travel time info
closest_road = list(range(0,len(df_only_geo)))

cluster_ids = list(range(0,len(df_only_geo)))
radius = 25000000 # metres
roads = gpd.read_file(test_data)
#print(roads)

for cluster in cluster_ids:
    point = Point(long_comm_c[cluster], lat_comm_c[cluster])
    buffer = point.buffer(radius)
    clipped_roads = gpd.clip(roads, buffer)

    # calculate distance for all lines
    distances = clipped_roads["geometry"].map(lambda line: point.distance(line))
    clipped_roads = clipped_roads.assign(Distance=distances)

    # get 4 roads closest to the point
    clipped_roads = clipped_roads.sort_values(by="Distance")
    closest_roads = clipped_roads.head(4)   
    closest_road[cluster]=closest_roads.loc[closest_roads.index.values[0], 'Distance']/1000
    
closest_road   
df_only_geo.insert(14, 'Closest Nat/Subnat Road (km)', closest_road)
df_only_geo

,index,Interview ID,Village,lat,long,"long,lat",age,sex,no_people_in_household,highest_grade_completed,...,toilet_type,Closest Nat/Subnat Road (km),Closest National Road (km),Land Cover,Luminance,Luminance Band 3,Luminance Band 2,Luminance Band 1,Travel Time (min),Precipitation (mm)
0,1,60645a36a2237b3f54690be5,Mabwe,0.595131,30.153128,"(30.15312827, 0.5951310726)",35.0,2.0,3.0,2.0,...,2.0,0.621699,4.495595,40.0,17.4412,29.0,17.0,15.0,24.0,1482.106567
1,8,60645af8a2237b3f5469146c,Bihondo,0.578531,30.170566,"(30.1705658, 0.5785313656)",45.0,1.0,7.0,0.0,...,2.0,1.330831,1.842590,40.0,17.2286,29.0,17.0,14.0,19.0,1482.106567
2,9,60645921a2237b3f5468ffc1,Okabi,3.125250,30.848807,"(30.84880735, 3.125250327)",64.0,2.0,4.0,0.0,...,2.0,5.073637,5.073637,40.0,18.5856,32.0,18.0,16.0,21.0,1345.285400
3,15,60d06f3edefc3909beb58988,Ngoro,0.196600,30.168300,"(30.1683, 0.1966)",28.0,1.0,4.0,4.0,...,2.0,6.338110,6.338110,126.0,15.2286,27.0,15.0,12.0,28.0,891.146545
4,22,60d06f50defc3909beb589a0,Kyigabo,0.264200,30.108400,"(30.1084, 0.2642)",35.0,1.0,5.0,2.0,...,2.0,0.355968,1.180557,40.0,17.6578,32.0,17.0,15.0,13.0,977.963806
5,25,60d06f53defc3909beb589a9,Kazinga,0.264200,30.108400,"(30.1084, 0.2642)",31.0,2.0,5.0,0.0,...,2.0,0.355968,1.180557,40.0,17.6578,32.0,17.0,15.0,13.0,977.963806
6,26,60d06f60defc3909beb589b1,Kitumba,0.589700,30.254900,"(30.2549, 0.5897)",37.0,1.0,4.0,1.0,...,2.0,3.752815,3.752815,20.0,15.8716,26.0,16.0,12.0,10.0,1247.092651
7,30,60d06f63defc3909beb589bc,Kakanju,0.542400,30.196500,"(30.1965, 0.5424)",28.0,2.0,3.0,2.0,...,2.0,2.745248,2.745248,30.0,16.1564,27.0,16.0,13.0,21.0,1482.106567
8,31,60d06f64defc3909beb589be,Kaburegye,0.542400,30.196500,"(30.1965, 0.5424)",45.0,2.0,4.0,0.0,...,2.0,2.745248,2.745248,30.0,16.1564,27.0,16.0,13.0,21.0,1482.106567
9,35,60d06f67defc3909beb589c6,Karambi,0.169900,30.078100,"(30.0781, 0.1699)",26.0,2.0,3.0,2.0,...,3.0,0.530190,0.530190,50.0,22.2314,41.0,21.0,20.0,0.0,1003.117432


# Proximity to rivers

In [42]:
test_data = "uganda_rivers_simple_2.shp"
df_river_predata_geo = gpd.read_file(test_data)
df_river_data_geo = df_river_predata_geo[['geometry']].copy()
df_river_data_geo

,geometry
0,"LINESTRING (3757552.537 408579.904, 3757537.84..."
1,"LINESTRING (3763978.994 408717.441, 3763972.98..."
2,"LINESTRING (3759235.909 408336.171, 3759227.33..."
3,"LINESTRING (3763615.652 408472.034, 3763608.74..."
4,"LINESTRING (3762505.349 408261.550, 3762493.10..."
...,...
32095,"LINESTRING (3317706.659 -115980.014, 3317703.5..."
32096,"LINESTRING (3312476.636 -115993.581, 3312483.8..."
32097,"LINESTRING (3328703.938 -116069.108, 3328684.6..."
32098,"LINESTRING (3312386.463 -116130.525, 3312392.9..."


In [43]:
test_data = "uganda_rivers_simple_2.shp"

# Instantiate list to hold travel time info
closest_river = list(range(0,len(df_only_geo)))

cluster_ids = list(range(0,len(df_only_geo)))
radius = 100000 # metres
rivers = gpd.read_file(test_data)
#print(rivers)
n=0
for cluster in cluster_ids:
    point = Point(long_comm_c[cluster], lat_comm_c[cluster])
    buffer = point.buffer(radius)
    clipped_rivers = gpd.clip(rivers, buffer)

    # calculate distance for all lines
    distances = clipped_rivers["geometry"].map(lambda line: point.distance(line))
    clipped_rivers = clipped_rivers.assign(Distance=distances)
    #display(clipped_rivers)
    
    # get 4 rivers closest to the point
    clipped_rivers = clipped_rivers.sort_values(by="Distance")
    #display('ojo', clipped_rivers)
    closest_rivers = clipped_rivers.head(4) 
    print('siii')
    
    closest_river[cluster]=closest_rivers.loc[closest_rivers.index.values[0], 'Distance']/1000
    display(closest_river[cluster])
    n=n+1
    print(cluster)
    
closest_river   
df_only_geo.insert(14, 'Closest Rivers (km)', closest_river)
df_only_geo

siii


0.8015318289484965

0
siii


0.8251464546121091

1
siii


...

0.6640650882858808

55
siii


0.6279806647845927

56


,index,Interview ID,Village,lat,long,"long,lat",age,sex,no_people_in_household,highest_grade_completed,...,Closest Rivers (km),Closest Nat/Subnat Road (km),Closest National Road (km),Land Cover,Luminance,Luminance Band 3,Luminance Band 2,Luminance Band 1,Travel Time (min),Precipitation (mm)
0,1,60645a36a2237b3f54690be5,Mabwe,0.595131,30.153128,"(30.15312827, 0.5951310726)",35.0,2.0,3.0,2.0,...,0.801532,0.621699,4.495595,40.0,17.4412,29.0,17.0,15.0,24.0,1482.106567
1,8,60645af8a2237b3f5469146c,Bihondo,0.578531,30.170566,"(30.1705658, 0.5785313656)",45.0,1.0,7.0,0.0,...,0.825146,1.330831,1.842590,40.0,17.2286,29.0,17.0,14.0,19.0,1482.106567
2,9,60645921a2237b3f5468ffc1,Okabi,3.125250,30.848807,"(30.84880735, 3.125250327)",64.0,2.0,4.0,0.0,...,1.018604,5.073637,5.073637,40.0,18.5856,32.0,18.0,16.0,21.0,1345.285400
3,15,60d06f3edefc3909beb58988,Ngoro,0.196600,30.168300,"(30.1683, 0.1966)",28.0,1.0,4.0,4.0,...,0.630554,6.338110,6.338110,126.0,15.2286,27.0,15.0,12.0,28.0,891.146545
4,22,60d06f50defc3909beb589a0,Kyigabo,0.264200,30.108400,"(30.1084, 0.2642)",35.0,1.0,5.0,2.0,...,0.221976,0.355968,1.180557,40.0,17.6578,32.0,17.0,15.0,13.0,977.963806
5,25,60d06f53defc3909beb589a9,Kazinga,0.264200,30.108400,"(30.1084, 0.2642)",31.0,2.0,5.0,0.0,...,0.221976,0.355968,1.180557,40.0,17.6578,32.0,17.0,15.0,13.0,977.963806
6,26,60d06f60defc3909beb589b1,Kitumba,0.589700,30.254900,"(30.2549, 0.5897)",37.0,1.0,4.0,1.0,...,0.422972,3.752815,3.752815,20.0,15.8716,26.0,16.0,12.0,10.0,1247.092651
7,30,60d06f63defc3909beb589bc,Kakanju,0.542400,30.196500,"(30.1965, 0.5424)",28.0,2.0,3.0,2.0,...,0.036051,2.745248,2.745248,30.0,16.1564,27.0,16.0,13.0,21.0,1482.106567
8,31,60d06f64defc3909beb589be,Kaburegye,0.542400,30.196500,"(30.1965, 0.5424)",45.0,2.0,4.0,0.0,...,0.036051,2.745248,2.745248,30.0,16.1564,27.0,16.0,13.0,21.0,1482.106567
9,35,60d06f67defc3909beb589c6,Karambi,0.169900,30.078100,"(30.0781, 0.1699)",26.0,2.0,3.0,2.0,...,1.375503,0.530190,0.530190,50.0,22.2314,41.0,21.0,20.0,0.0,1003.117432


# Nearest distance to transmission substations
##### https://energydata.info/dataset/transmission-substations-2016-operational
##### https://energydata.info/dataset/distribution-substations-2016

In [44]:
#Read substations geographic data and extract relevant parameters
sub_geo = "transmission_substations_operational.geojson"
df_sub_predata_geo = gpd.read_file(sub_geo)
df_sub_data_geo = df_sub_predata_geo[['OBJECTID', 'Name', 'Voltage', 'Latitude', 'Longitude', 'geometry']].copy()

In [45]:
#List of projected coordinates for communities in Stephi's data set (df_only_geo)

lat_comm = list(df_only_geo['lat'])
long_comm = list(df_only_geo['long'])

long_comm_c = list(range(0,len(df_only_geo)))
lat_comm_c = list(range(0,len(df_only_geo)))

i=0

proj_wgs84 = pyproj.Proj(init="epsg:4326")
proj_gk4 = pyproj.Proj(init="epsg:3857")

for (long_comm,lat_comm) in zip(long_comm,lat_comm):
    long_comm_c[i], lat_comm_c[i] = pyproj.transform(proj_wgs84, proj_gk4, long_comm, lat_comm)
    i=i+1
    
#display(long_comm_c)
#display(lat_comm_c)

#List of projected coordinates from substations in Substation data (df_sub_data_geo)

lat_sub = list(df_sub_data_geo['Latitude'])
long_sub = list(df_sub_data_geo['Longitude'])

long_sub_c = list(range(0,len(df_sub_data_geo)))
lat_sub_c = list(range(0,len(df_sub_data_geo)))

i=0

proj_wgs84 = pyproj.Proj(init="epsg:4326")
proj_gk4 = pyproj.Proj(init="epsg:3857")

for (long_sub,lat_sub) in zip(long_sub,lat_sub):
    long_sub_c[i], lat_sub_c[i] = pyproj.transform(proj_wgs84, proj_gk4, long_sub, lat_sub)
    i=i+1
    
#display(long_sub_c)
#display(lat_sub_c)

In [46]:
## Get the distance from each community point to the nearest substation

d = {'long_c (comm)':long_comm_c,'lat_c (comm)':lat_comm_c}
communities = gpd.GeoDataFrame(d)

d2 = {'long_c (sub)':long_sub_c,'lat_c (sub)':lat_sub_c}
substations = gpd.GeoDataFrame(d2)

# Create a BallTree 
tree = BallTree(substations[['long_c (sub)', 'lat_c (sub)']].values, leaf_size=2)

# Query the BallTree on each feature from 'communities' to find the distance
# to the nearest 'substation' and its id
communities['distance_nearest'], communities['id_nearest'] = tree.query(
    communities[['long_c (comm)', 'lat_c (comm)']].values, # The input array for the query
    k=1, # The number of nearest neighbors
)
#display(substations)
#display(communities)


#Look for the closest substation for each observation
i=0
while i < len(communities):
    #print('----------soyi', i)
    id_nearest = communities.loc[communities.index.values[i], 'id_nearest']
    #print(id_nearest)
    j=0
    while j < len(df_sub_data_geo):
        #print('-soyj', j)
        if id_nearest == j:
            communities.loc[communities.index.values[i], 'Substation id'] = df_sub_data_geo.loc[df_sub_data_geo.index.values[j], 'OBJECTID']
            break
        else:
            j=j+1        
    i=i+1       

k=0

while k < len(communities):
    communities.loc[communities.index.values[k], 'distance_nearest'] =  communities.loc[communities.index.values[k], 'distance_nearest']/1000
    k=k+1
#display(communities)

distance_substation = communities['distance_nearest'].tolist()
   
df_only_geo.insert(14, 'Closest Trans. Substation (km)', distance_substation)
#print(type(df_only_geo.loc[3,'Closest Trans. Substation (km)']))
df_only_geo

,index,Interview ID,Village,lat,long,"long,lat",age,sex,no_people_in_household,highest_grade_completed,...,Closest Rivers (km),Closest Nat/Subnat Road (km),Closest National Road (km),Land Cover,Luminance,Luminance Band 3,Luminance Band 2,Luminance Band 1,Travel Time (min),Precipitation (mm)
0,1,60645a36a2237b3f54690be5,Mabwe,0.595131,30.153128,"(30.15312827, 0.5951310726)",35.0,2.0,3.0,2.0,...,0.801532,0.621699,4.495595,40.0,17.4412,29.0,17.0,15.0,24.0,1482.106567
1,8,60645af8a2237b3f5469146c,Bihondo,0.578531,30.170566,"(30.1705658, 0.5785313656)",45.0,1.0,7.0,0.0,...,0.825146,1.330831,1.842590,40.0,17.2286,29.0,17.0,14.0,19.0,1482.106567
2,9,60645921a2237b3f5468ffc1,Okabi,3.125250,30.848807,"(30.84880735, 3.125250327)",64.0,2.0,4.0,0.0,...,1.018604,5.073637,5.073637,40.0,18.5856,32.0,18.0,16.0,21.0,1345.285400
3,15,60d06f3edefc3909beb58988,Ngoro,0.196600,30.168300,"(30.1683, 0.1966)",28.0,1.0,4.0,4.0,...,0.630554,6.338110,6.338110,126.0,15.2286,27.0,15.0,12.0,28.0,891.146545
4,22,60d06f50defc3909beb589a0,Kyigabo,0.264200,30.108400,"(30.1084, 0.2642)",35.0,1.0,5.0,2.0,...,0.221976,0.355968,1.180557,40.0,17.6578,32.0,17.0,15.0,13.0,977.963806
5,25,60d06f53defc3909beb589a9,Kazinga,0.264200,30.108400,"(30.1084, 0.2642)",31.0,2.0,5.0,0.0,...,0.221976,0.355968,1.180557,40.0,17.6578,32.0,17.0,15.0,13.0,977.963806
6,26,60d06f60defc3909beb589b1,Kitumba,0.589700,30.254900,"(30.2549, 0.5897)",37.0,1.0,4.0,1.0,...,0.422972,3.752815,3.752815,20.0,15.8716,26.0,16.0,12.0,10.0,1247.092651
7,30,60d06f63defc3909beb589bc,Kakanju,0.542400,30.196500,"(30.1965, 0.5424)",28.0,2.0,3.0,2.0,...,0.036051,2.745248,2.745248,30.0,16.1564,27.0,16.0,13.0,21.0,1482.106567
8,31,60d06f64defc3909beb589be,Kaburegye,0.542400,30.196500,"(30.1965, 0.5424)",45.0,2.0,4.0,0.0,...,0.036051,2.745248,2.745248,30.0,16.1564,27.0,16.0,13.0,21.0,1482.106567
9,35,60d06f67defc3909beb589c6,Karambi,0.169900,30.078100,"(30.0781, 0.1699)",26.0,2.0,3.0,2.0,...,1.375503,0.530190,0.530190,50.0,22.2314,41.0,21.0,20.0,0.0,1003.117432


# Nearest distance to distribution substations
##### https://energydata.info/dataset/transmission-substations-2016-operational
##### https://energydata.info/dataset/distribution-substations-2016

In [47]:
#Read substations geographic data and extract relevant parameters
sub_geo = "distribution_substations_2016.geojson"
df_sub_predata_geo = gpd.read_file(sub_geo)
df_sub_data_geo = df_sub_predata_geo[['OBJECTID', 'NAME', 'Voltage', 'geometry']].copy()
df_sub_data_geo

,OBJECTID,NAME,Voltage,geometry
0,1,KALIRO,33/11,POINT (33.50343 0.88750)
1,2,KITGUM,33/11,POINT (32.88034 3.29938)
2,3,GULU,33/11,POINT (32.30152 2.75883)
3,4,FORT PORTAL,33/11,POINT (30.30218 0.65241)
4,5,HOIMA,33/11,POINT (31.37300 1.42074)
...,...,...,...,...
71,72,MOTORMART,11/11,POINT (32.58315 0.31202)
72,73,KITANTE,33/11,POINT (32.59403 0.31757)
73,74,RWANDA,30/11,POINT (29.74044 -1.34065)
74,75,QUEENSWAY 2,33/11,POINT (32.57721 0.30513)


In [48]:
#Conversion of 'geometry' column in distribution_substations_2016.geojson into a dataframe
list_of_tuples=list(range(0,len(df_sub_data_geo)))
i=0
while i < len(df_sub_data_geo):
    point = df_sub_data_geo.loc[df_sub_data_geo.index.values[i], 'geometry'].coords
    list_of_tuples[i]=list(point)
    i=i+1

points = gpd.GeoDataFrame(list_of_tuples, columns=["x"])
points2 = points['x'].tolist()
points_sub = gpd.GeoDataFrame(points2, columns=["Longitude","Latitude"])
#points_sub

In [49]:
#List of projected coordinates for communities in Stephi's data set (df_only_geo)

lat_comm = list(df_only_geo['lat'])
long_comm = list(df_only_geo['long'])

long_comm_c = list(range(0,len(df_only_geo)))
lat_comm_c = list(range(0,len(df_only_geo)))

i=0

proj_wgs84 = pyproj.Proj(init="epsg:4326")
proj_gk4 = pyproj.Proj(init="epsg:3857")

for (long_comm,lat_comm) in zip(long_comm,lat_comm):
    long_comm_c[i], lat_comm_c[i] = pyproj.transform(proj_wgs84, proj_gk4, long_comm, lat_comm)
    i=i+1
    
#display(long_comm_c)
#display(lat_comm_c)

#List of projected coordinates from substations in Substation data (df_sub_data_geo)

lat_sub = list(points_sub['Latitude'])
long_sub = list(points_sub['Longitude'])

long_sub_c = list(range(0,len(points_sub)))
lat_sub_c = list(range(0,len(points_sub)))

i=0

proj_wgs84 = pyproj.Proj(init="epsg:4326")
proj_gk4 = pyproj.Proj(init="epsg:3857")

for (long_sub,lat_sub) in zip(long_sub,lat_sub):
    long_sub_c[i], lat_sub_c[i] = pyproj.transform(proj_wgs84, proj_gk4, long_sub, lat_sub)
    i=i+1
    
#display(long_sub_c)
#display(lat_sub_c)

In [50]:
## Get the distance from each community point to the nearest substation

d = {'long_c (comm)':long_comm_c,'lat_c (comm)':lat_comm_c}
communities = gpd.GeoDataFrame(d)

d2 = {'long_c (sub)':long_sub_c,'lat_c (sub)':lat_sub_c}
substations = gpd.GeoDataFrame(d2)

# Create a BallTree 
tree = BallTree(substations[['long_c (sub)', 'lat_c (sub)']].values, leaf_size=2)

# Query the BallTree on each feature from 'communities' to find the distance
# to the nearest 'substation' and its id
communities['distance_nearest'], communities['id_nearest'] = tree.query(
    communities[['long_c (comm)', 'lat_c (comm)']].values, # The input array for the query
    k=1, # The number of nearest neighbors
)
#display(substations)
#display(communities)


#Look for the closest substation for each observation
i=0
while i < len(communities):
    #print('----------soyi', i)
    id_nearest = communities.loc[communities.index.values[i], 'id_nearest']
    #print(id_nearest)
    j=0
    while j < len(df_sub_data_geo):
        #print('-soyj', j)
        if id_nearest == j:
            communities.loc[communities.index.values[i], 'Substation id'] = df_sub_data_geo.loc[df_sub_data_geo.index.values[j], 'OBJECTID']
            break
        else:
            j=j+1        
    i=i+1       

k=0

while k < len(communities):
    communities.loc[communities.index.values[k], 'distance_nearest'] =  communities.loc[communities.index.values[k], 'distance_nearest']/1000
    k=k+1
#display(communities)

distance_substation = communities['distance_nearest'].tolist()
   
df_only_geo.insert(14, 'Closest Distr. Substation (km)', distance_substation)
#print(type(df_only_geo.loc[3,'Closest Trans. Substation (km)']))
df_only_geo

,index,Interview ID,Village,lat,long,"long,lat",age,sex,no_people_in_household,highest_grade_completed,...,Closest Rivers (km),Closest Nat/Subnat Road (km),Closest National Road (km),Land Cover,Luminance,Luminance Band 3,Luminance Band 2,Luminance Band 1,Travel Time (min),Precipitation (mm)
0,1,60645a36a2237b3f54690be5,Mabwe,0.595131,30.153128,"(30.15312827, 0.5951310726)",35.0,2.0,3.0,2.0,...,0.801532,0.621699,4.495595,40.0,17.4412,29.0,17.0,15.0,24.0,1482.106567
1,8,60645af8a2237b3f5469146c,Bihondo,0.578531,30.170566,"(30.1705658, 0.5785313656)",45.0,1.0,7.0,0.0,...,0.825146,1.330831,1.842590,40.0,17.2286,29.0,17.0,14.0,19.0,1482.106567
2,9,60645921a2237b3f5468ffc1,Okabi,3.125250,30.848807,"(30.84880735, 3.125250327)",64.0,2.0,4.0,0.0,...,1.018604,5.073637,5.073637,40.0,18.5856,32.0,18.0,16.0,21.0,1345.285400
3,15,60d06f3edefc3909beb58988,Ngoro,0.196600,30.168300,"(30.1683, 0.1966)",28.0,1.0,4.0,4.0,...,0.630554,6.338110,6.338110,126.0,15.2286,27.0,15.0,12.0,28.0,891.146545
4,22,60d06f50defc3909beb589a0,Kyigabo,0.264200,30.108400,"(30.1084, 0.2642)",35.0,1.0,5.0,2.0,...,0.221976,0.355968,1.180557,40.0,17.6578,32.0,17.0,15.0,13.0,977.963806
5,25,60d06f53defc3909beb589a9,Kazinga,0.264200,30.108400,"(30.1084, 0.2642)",31.0,2.0,5.0,0.0,...,0.221976,0.355968,1.180557,40.0,17.6578,32.0,17.0,15.0,13.0,977.963806
6,26,60d06f60defc3909beb589b1,Kitumba,0.589700,30.254900,"(30.2549, 0.5897)",37.0,1.0,4.0,1.0,...,0.422972,3.752815,3.752815,20.0,15.8716,26.0,16.0,12.0,10.0,1247.092651
7,30,60d06f63defc3909beb589bc,Kakanju,0.542400,30.196500,"(30.1965, 0.5424)",28.0,2.0,3.0,2.0,...,0.036051,2.745248,2.745248,30.0,16.1564,27.0,16.0,13.0,21.0,1482.106567
8,31,60d06f64defc3909beb589be,Kaburegye,0.542400,30.196500,"(30.1965, 0.5424)",45.0,2.0,4.0,0.0,...,0.036051,2.745248,2.745248,30.0,16.1564,27.0,16.0,13.0,21.0,1482.106567
9,35,60d06f67defc3909beb589c6,Karambi,0.169900,30.078100,"(30.0781, 0.1699)",26.0,2.0,3.0,2.0,...,1.375503,0.530190,0.530190,50.0,22.2314,41.0,21.0,20.0,0.0,1003.117432


# Proximity to Forests (the nearest)
##### https://pearlgeoportal.com/sector/environment/shapefiles
##### https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/BEVMRY

In [51]:
#Get coordinates from the polygons
import pyproj
from shapely.ops import transform

project = pyproj.Transformer.from_proj(
    pyproj.Proj(init='epsg:4326'), # source coordinate system
    pyproj.Proj(init='epsg:3857')) # destination coordinate system

# g1 is a shapley Polygon
forest_geo = "WDPA_June2017_UGA-shapefile-polygons.shp"
df_forest_predata_geo = gpd.read_file(forest_geo)
g1 = df_forest_predata_geo[['geometry']].copy()
#display(g1)

lista_poly= list(range(0,len(g1)))
  
for i in lista_poly:
    lista_poly[i] = transform(project.transform, g1.loc[g1.index.values[i], 'geometry'])  # apply projection

geometry2 = gpd.GeoDataFrame(lista_poly, columns=["geometry"])
geometry2

,geometry
0,"POLYGON ((3670985.074 182027.620, 3670242.573 ..."
1,"POLYGON ((3742551.262 84876.938, 3742285.208 8..."
2,"POLYGON ((3835378.359 155846.095, 3835100.060 ..."
3,"POLYGON ((3373487.764 10625.708, 3371907.451 7..."
4,"MULTIPOLYGON (((3626806.821 382543.693, 362669..."
...,...
698,"MULTIPOLYGON (((3840245.247 178135.453, 384024..."
699,"MULTIPOLYGON (((3301136.273 -66092.072, 330113..."
700,"MULTIPOLYGON (((3301136.062 -66091.564, 330113..."
701,"POLYGON ((3304889.778 -97938.284, 3305028.927 ..."


In [52]:
#List of projected coordinates for communities in Stephi's data set (df_only_geo)

lat_comm = list(df_only_geo['lat'])
long_comm = list(df_only_geo['long'])

#print(type(lat_comm[3])) 

long_comm_c = list(range(0,len(df_only_geo)))
lat_comm_c = list(range(0,len(df_only_geo)))

i=0

proj_wgs84 = pyproj.Proj(init="epsg:4326")
proj_gk4 = pyproj.Proj(init="epsg:3857")

for (long_comm,lat_comm) in zip(long_comm,lat_comm):
    long_comm_c[i], lat_comm_c[i] = pyproj.transform(proj_wgs84, proj_gk4, long_comm, lat_comm)
    i=i+1
    
#display(long_comm_c)
#display(lat_comm_c)

#List of projected coordinates from substations in Substation data (df_sub_data_geo)

#lat_sub = list(df_sub_data_geo['Latitude'])
#long_sub = list(df_sub_data_geo['Longitude'])

#long_sub_c = list(range(0,len(df_sub_data_geo)))
#lat_sub_c = list(range(0,len(df_sub_data_geo)))

#i=0

#proj_wgs84 = pyproj.Proj(init="epsg:4326")
#proj_gk4 = pyproj.Proj(init="epsg:3857")

#for (long_sub,lat_sub) in zip(long_sub,lat_sub):
    #long_sub_c[i], lat_sub_c[i] = pyproj.transform(proj_wgs84, proj_gk4, long_sub, lat_sub)
    #i=i+1
    
#display(long_sub_c)
#display(lat_sub_c)
#print(type(lat_comm_c[3])) 
#lat_comm

In [53]:
# Instantiate list to hold travel time info
closest_forest = list(range(0,len(df_only_geo)))

cluster_ids = list(range(0,len(df_only_geo)))
radius = 25000000 # metres

for cluster in cluster_ids:
    point = Point(long_comm_c[cluster], lat_comm_c[cluster])
    buffer = point.buffer(radius)
    clipped_forests = gpd.clip(geometry2, buffer)
    
    # calculate distance for all lines
    distances = clipped_forests["geometry"].map(lambda line: point.distance(line))
    clipped_forests = clipped_forests.assign(Distance=distances)

    # get 4 roads closest to the point
    clipped_forests = clipped_forests.sort_values(by="Distance")
    closest_forests = clipped_forests.head(4)  
    
    closest_forest[cluster]=closest_forests.loc[closest_forests.index.values[0], 'Distance']/1000
     
df_only_geo.insert(14, 'Closest forest (km)', closest_forest)
df_only_geo

,index,Interview ID,Village,lat,long,"long,lat",age,sex,no_people_in_household,highest_grade_completed,...,Closest Rivers (km),Closest Nat/Subnat Road (km),Closest National Road (km),Land Cover,Luminance,Luminance Band 3,Luminance Band 2,Luminance Band 1,Travel Time (min),Precipitation (mm)
0,1,60645a36a2237b3f54690be5,Mabwe,0.595131,30.153128,"(30.15312827, 0.5951310726)",35.0,2.0,3.0,2.0,...,0.801532,0.621699,4.495595,40.0,17.4412,29.0,17.0,15.0,24.0,1482.106567
1,8,60645af8a2237b3f5469146c,Bihondo,0.578531,30.170566,"(30.1705658, 0.5785313656)",45.0,1.0,7.0,0.0,...,0.825146,1.330831,1.842590,40.0,17.2286,29.0,17.0,14.0,19.0,1482.106567
2,9,60645921a2237b3f5468ffc1,Okabi,3.125250,30.848807,"(30.84880735, 3.125250327)",64.0,2.0,4.0,0.0,...,1.018604,5.073637,5.073637,40.0,18.5856,32.0,18.0,16.0,21.0,1345.285400
3,15,60d06f3edefc3909beb58988,Ngoro,0.196600,30.168300,"(30.1683, 0.1966)",28.0,1.0,4.0,4.0,...,0.630554,6.338110,6.338110,126.0,15.2286,27.0,15.0,12.0,28.0,891.146545
4,22,60d06f50defc3909beb589a0,Kyigabo,0.264200,30.108400,"(30.1084, 0.2642)",35.0,1.0,5.0,2.0,...,0.221976,0.355968,1.180557,40.0,17.6578,32.0,17.0,15.0,13.0,977.963806
5,25,60d06f53defc3909beb589a9,Kazinga,0.264200,30.108400,"(30.1084, 0.2642)",31.0,2.0,5.0,0.0,...,0.221976,0.355968,1.180557,40.0,17.6578,32.0,17.0,15.0,13.0,977.963806
6,26,60d06f60defc3909beb589b1,Kitumba,0.589700,30.254900,"(30.2549, 0.5897)",37.0,1.0,4.0,1.0,...,0.422972,3.752815,3.752815,20.0,15.8716,26.0,16.0,12.0,10.0,1247.092651
7,30,60d06f63defc3909beb589bc,Kakanju,0.542400,30.196500,"(30.1965, 0.5424)",28.0,2.0,3.0,2.0,...,0.036051,2.745248,2.745248,30.0,16.1564,27.0,16.0,13.0,21.0,1482.106567
8,31,60d06f64defc3909beb589be,Kaburegye,0.542400,30.196500,"(30.1965, 0.5424)",45.0,2.0,4.0,0.0,...,0.036051,2.745248,2.745248,30.0,16.1564,27.0,16.0,13.0,21.0,1482.106567
9,35,60d06f67defc3909beb589c6,Karambi,0.169900,30.078100,"(30.0781, 0.1699)",26.0,2.0,3.0,2.0,...,1.375503,0.530190,0.530190,50.0,22.2314,41.0,21.0,20.0,0.0,1003.117432


# Proximity to Forests (10km radius, smallest area, nearest)
##### https://pearlgeoportal.com/sector/environment/shapefiles
##### https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/BEVMRY

In [54]:
#Get coordinates from the polygons
import pyproj
from shapely.ops import transform

project = pyproj.Transformer.from_proj(
    pyproj.Proj(init='epsg:4326'), # source coordinate system
    pyproj.Proj(init='epsg:3857')) # destination coordinate system

# g1 is a shapley Polygon
forest_geo = "WDPA_June2017_UGA-shapefile-polygons.shp"
df_forest_predata_geo = gpd.read_file(forest_geo)
g1 = df_forest_predata_geo[['geometry', 'NAME', 'REP_AREA', 'GIS_AREA']].copy()
#display(g1)

lista_poly= list(range(0,len(g1)))
lista_area= list(range(0,len(g1)))
  
for i in lista_poly:
    lista_poly[i] = transform(project.transform, g1.loc[g1.index.values[i], 'geometry'])  # apply projection
    lista_area[i] = (lista_poly[i].area)/1000000
#print(lista_g2)

#geometry2 = gpd.GeoDataFrame(lista_poly, columns=["geometry"])
#display(geometry2)
#areas = gpd.GeoDataFrame(lista_area, columns=["Area (km2)"])
#display(areas)

d = {'geometry':lista_poly,'Area (km2)':lista_area}
geometry2 = gpd.GeoDataFrame(d, columns=['geometry','Area (km2)'])
geometry2

,geometry,Area (km2)
0,"POLYGON ((3670985.074 182027.620, 3670242.573 ...",3.725850
1,"POLYGON ((3742551.262 84876.938, 3742285.208 8...",0.156182
2,"POLYGON ((3835378.359 155846.095, 3835100.060 ...",0.060192
3,"POLYGON ((3373487.764 10625.708, 3371907.451 7...",580.719617
4,"MULTIPOLYGON (((3626806.821 382543.693, 362669...",0.047166
...,...,...
698,"MULTIPOLYGON (((3840245.247 178135.453, 384024...",2170.423571
699,"MULTIPOLYGON (((3301136.273 -66092.072, 330113...",2535.604510
700,"MULTIPOLYGON (((3301136.062 -66091.564, 330113...",2112.739783
701,"POLYGON ((3304889.778 -97938.284, 3305028.927 ...",329.537438


In [55]:
# Instantiate list to hold travel time info
closest_forest = list(range(0,len(df_only_geo)))

cluster_ids = list(range(0,len(df_only_geo)))
#radius = 25000000000 # metres
count = 0
i=0
walk=10000

for cluster in cluster_ids:
    point = Point(long_comm_c[cluster], lat_comm_c[cluster])  #we have already defined 'lat_comm_c' before, no need now.
    #buffer = point.buffer(radius)
    clipped_forests = geometry2
    
    # calculate distance for all lines
    distances = clipped_forests["geometry"].map(lambda line: point.distance(line))
    clipped_forests = clipped_forests.assign(Distance=distances)
    clipped_forests_ = clipped_forests
    
    # get 4 roads closest to the point
    clipped_forests = clipped_forests.sort_values(by="Distance")
    closest_forests = clipped_forests.head(703)  
    
    while closest_forests.loc[closest_forests.index.values[i], 'Distance'] < walk:
        count = count + 1
        i=i+1
    selected = list(range(0,count))
   
    newdf =closest_forests.iloc[0:count]
    #display(newdf)

    index = newdf['Area (km2)'].idxmin()
    #print(index)
    closest_forest[cluster]=clipped_forests_.loc[clipped_forests_.index[index], 'Distance']/1000
     
df_only_geo.insert(14, 'Closest smallest forest radius (km)', closest_forest)
df_only_geo

,index,Interview ID,Village,lat,long,"long,lat",age,sex,no_people_in_household,highest_grade_completed,...,Closest Rivers (km),Closest Nat/Subnat Road (km),Closest National Road (km),Land Cover,Luminance,Luminance Band 3,Luminance Band 2,Luminance Band 1,Travel Time (min),Precipitation (mm)
0,1,60645a36a2237b3f54690be5,Mabwe,0.595131,30.153128,"(30.15312827, 0.5951310726)",35.0,2.0,3.0,2.0,...,0.801532,0.621699,4.495595,40.0,17.4412,29.0,17.0,15.0,24.0,1482.106567
1,8,60645af8a2237b3f5469146c,Bihondo,0.578531,30.170566,"(30.1705658, 0.5785313656)",45.0,1.0,7.0,0.0,...,0.825146,1.330831,1.842590,40.0,17.2286,29.0,17.0,14.0,19.0,1482.106567
2,9,60645921a2237b3f5468ffc1,Okabi,3.125250,30.848807,"(30.84880735, 3.125250327)",64.0,2.0,4.0,0.0,...,1.018604,5.073637,5.073637,40.0,18.5856,32.0,18.0,16.0,21.0,1345.285400
3,15,60d06f3edefc3909beb58988,Ngoro,0.196600,30.168300,"(30.1683, 0.1966)",28.0,1.0,4.0,4.0,...,0.630554,6.338110,6.338110,126.0,15.2286,27.0,15.0,12.0,28.0,891.146545
4,22,60d06f50defc3909beb589a0,Kyigabo,0.264200,30.108400,"(30.1084, 0.2642)",35.0,1.0,5.0,2.0,...,0.221976,0.355968,1.180557,40.0,17.6578,32.0,17.0,15.0,13.0,977.963806
5,25,60d06f53defc3909beb589a9,Kazinga,0.264200,30.108400,"(30.1084, 0.2642)",31.0,2.0,5.0,0.0,...,0.221976,0.355968,1.180557,40.0,17.6578,32.0,17.0,15.0,13.0,977.963806
6,26,60d06f60defc3909beb589b1,Kitumba,0.589700,30.254900,"(30.2549, 0.5897)",37.0,1.0,4.0,1.0,...,0.422972,3.752815,3.752815,20.0,15.8716,26.0,16.0,12.0,10.0,1247.092651
7,30,60d06f63defc3909beb589bc,Kakanju,0.542400,30.196500,"(30.1965, 0.5424)",28.0,2.0,3.0,2.0,...,0.036051,2.745248,2.745248,30.0,16.1564,27.0,16.0,13.0,21.0,1482.106567
8,31,60d06f64defc3909beb589be,Kaburegye,0.542400,30.196500,"(30.1965, 0.5424)",45.0,2.0,4.0,0.0,...,0.036051,2.745248,2.745248,30.0,16.1564,27.0,16.0,13.0,21.0,1482.106567
9,35,60d06f67defc3909beb589c6,Karambi,0.169900,30.078100,"(30.0781, 0.1699)",26.0,2.0,3.0,2.0,...,1.375503,0.530190,0.530190,50.0,22.2314,41.0,21.0,20.0,0.0,1003.117432


# Deforestation 

In [56]:
# Filepaths to raster
deforestation_raster = "deforestation_uganda_2000_2021(2).tif"
#https://storage.googleapis.com/earthenginepartners-hansen/GFC-2021-v1.9/download.html   (0 to 20, with 0 'no loss')

In [57]:
deforestation = list(range(0,len(df_only_geo)))

walk = 10000 #Distance in meters that a person walks to get firewood
distance_x_pixel = 30 #meters (1 arc second)
distance_y_pixel = 30 #meters (1 arc second)
distance_pixel = (distance_x_pixel + distance_y_pixel) / 2
pixels = round(walk / distance_pixel)
deforestation2 = list(range(0,pixels*2*pixels*2))
total_pixels = pixels*2*pixels*2
value_total_pixels = 20 * total_pixels

#print(len(deforestation2))
#print('Distance pixel prom:', distance_pixel, 'Pixels:', pixels)

# Read in travel time data
deforestation_gdata = gdal.Open(deforestation_raster)
deforestation_gt = deforestation_gdata.GetGeoTransform()
deforestation_data = deforestation_gdata.ReadAsArray().astype(np.float)
#display(deforestation_gt)

# Fake list of cluster_ids
cluster_ids = list(range(0,len(df_only_geo)))

#Lists for getting the data of around 10 km of the point
pixels_list = list(range(0,pixels*2))


# Fake list of mean coordinates
mean_coord = list(df_only_geo['long,lat'])
#print(type(mean_coord[0]))

# Get information about each cluster. For reference: in the geotransform loaded from GDAL,
# [0] = top left x, [1] = w-e pixel resolution, [2] = 0, [3] = top left y, [4] = 0, [5] = n-s pixel resolution

#(28.76875, 0.00025000000000000006, 0.0, 4.9975, 0.0, -0.00025

In [58]:
for cluster in cluster_ids:
    # Get the wind value at the mean point of the cluster
    x = int((mean_coord[cluster][0] - deforestation_gt[0])/deforestation_gt[1])
    #print(x)
    y = int((mean_coord[cluster][1] - deforestation_gt[3])/deforestation_gt[5])
    #print(y)

    x_start = x - pixels
    y_start = y - pixels
    
    k = 0
    kk = 0
    i=0
    j=0
    while i < len(pixels_list):
        x_start = x_start + 1
        #print ('x', x_start)
        j=0
        while j < len(pixels_list):
            deforestation2[k] = deforestation_data[y_start+j, x_start]        
            #print(deforestation2[k])
            #print(y_start+j, x_start)  
            #print(k, j)
            k = k+1
            j = j+1
            #print('termine', deforestation2[k])
        
        i = i+1   
    deforestation[cluster] = sum(deforestation2)/value_total_pixels
    
    #print('this', sum(deforestation2))
    #print('this', statistics.mean(deforestation2))
    

df_only_geo.insert(14, 'Grade of Deforestation', deforestation)
df_only_geo

,index,Interview ID,Village,lat,long,"long,lat",age,sex,no_people_in_household,highest_grade_completed,...,Closest Rivers (km),Closest Nat/Subnat Road (km),Closest National Road (km),Land Cover,Luminance,Luminance Band 3,Luminance Band 2,Luminance Band 1,Travel Time (min),Precipitation (mm)
0,1,60645a36a2237b3f54690be5,Mabwe,0.595131,30.153128,"(30.15312827, 0.5951310726)",35.0,2.0,3.0,2.0,...,0.801532,0.621699,4.495595,40.0,17.4412,29.0,17.0,15.0,24.0,1482.106567
1,8,60645af8a2237b3f5469146c,Bihondo,0.578531,30.170566,"(30.1705658, 0.5785313656)",45.0,1.0,7.0,0.0,...,0.825146,1.330831,1.842590,40.0,17.2286,29.0,17.0,14.0,19.0,1482.106567
2,9,60645921a2237b3f5468ffc1,Okabi,3.125250,30.848807,"(30.84880735, 3.125250327)",64.0,2.0,4.0,0.0,...,1.018604,5.073637,5.073637,40.0,18.5856,32.0,18.0,16.0,21.0,1345.285400
3,15,60d06f3edefc3909beb58988,Ngoro,0.196600,30.168300,"(30.1683, 0.1966)",28.0,1.0,4.0,4.0,...,0.630554,6.338110,6.338110,126.0,15.2286,27.0,15.0,12.0,28.0,891.146545
4,22,60d06f50defc3909beb589a0,Kyigabo,0.264200,30.108400,"(30.1084, 0.2642)",35.0,1.0,5.0,2.0,...,0.221976,0.355968,1.180557,40.0,17.6578,32.0,17.0,15.0,13.0,977.963806
5,25,60d06f53defc3909beb589a9,Kazinga,0.264200,30.108400,"(30.1084, 0.2642)",31.0,2.0,5.0,0.0,...,0.221976,0.355968,1.180557,40.0,17.6578,32.0,17.0,15.0,13.0,977.963806
6,26,60d06f60defc3909beb589b1,Kitumba,0.589700,30.254900,"(30.2549, 0.5897)",37.0,1.0,4.0,1.0,...,0.422972,3.752815,3.752815,20.0,15.8716,26.0,16.0,12.0,10.0,1247.092651
7,30,60d06f63defc3909beb589bc,Kakanju,0.542400,30.196500,"(30.1965, 0.5424)",28.0,2.0,3.0,2.0,...,0.036051,2.745248,2.745248,30.0,16.1564,27.0,16.0,13.0,21.0,1482.106567
8,31,60d06f64defc3909beb589be,Kaburegye,0.542400,30.196500,"(30.1965, 0.5424)",45.0,2.0,4.0,0.0,...,0.036051,2.745248,2.745248,30.0,16.1564,27.0,16.0,13.0,21.0,1482.106567
9,35,60d06f67defc3909beb589c6,Karambi,0.169900,30.078100,"(30.0781, 0.1699)",26.0,2.0,3.0,2.0,...,1.375503,0.530190,0.530190,50.0,22.2314,41.0,21.0,20.0,0.0,1003.117432


In [59]:
print(value_total_pixels)

8871120


In [60]:
#Get the final table 
df_result = pd.concat([df_only_geo, df_new_data_order], axis=1)
df_result.to_excel("00._output_v3_Observations_falses.xlsx")
df_result.head(1)

,index,Interview ID,Village,lat,long,"long,lat",age,sex,no_people_in_household,highest_grade_completed,...,Unburden,Comfort,Water Security,Medical treatment,Appeal (senses),Personal performance,Aspiration,Time management,Income,Reliability
0,1,60645a36a2237b3f54690be5,Mabwe,0.595131,30.153128,"(30.15312827, 0.5951310726)",35.0,2.0,3.0,2.0,...,1,1,0,0,1,0,1,0,1,1


# Only presence of forest (All forests considered)

In [61]:
# Filepaths to raster
presence_forest1_raster = "only_forest_uganda.tif"
#https://lcviewer.vito.be/2016/Uganda
#https://lcviewer.vito.be/download   (0 to 100, with 0 'no forest')

In [62]:
presence_forest1 = list(range(0,len(df_only_geo)))

walk = 10000 #Distance in meters that a person walks to get firewood
distance_x_pixel = 100 #meters(measured in QGIS = 110.383)
distance_y_pixel = 100 #meters (measured in QGIS = 109.841)
distance_pixel = (distance_x_pixel + distance_y_pixel) / 2
pixels = round(walk / distance_pixel)
presence_forest12 = list(range(0,pixels*2*pixels*2))
total_pixels = pixels*2*pixels*2
value_total_pixels = 100 * total_pixels

#print(len(presence_forest2))
#print('Distance pixel prom:', distance_pixel, 'Pixels:', pixels)

# Read in travel time data
presence_forest1_gdata = gdal.Open(presence_forest1_raster)
presence_forest1_gt = presence_forest1_gdata.GetGeoTransform()
presence_forest1_data = presence_forest1_gdata.ReadAsArray().astype(np.float)
#display(presence_forest_gt)

# Fake list of cluster_ids
cluster_ids = list(range(0,len(df_only_geo)))

#Lists for getting the data of around 10 km of the point
pixels_list = list(range(0,pixels*2))


# Fake list of mean coordinates
mean_coord = list(df_only_geo['long,lat'])
#print(type(mean_coord[0]))

# Get information about each cluster. For reference: in the geotransform loaded from GDAL,
# [0] = top left x, [1] = w-e pixel resolution, [2] = 0, [3] = top left y, [4] = 0, [5] = n-s pixel resolution

#(28.76875, 0.00025000000000000006, 0.0, 4.9975, 0.0, -0.00025)

In [63]:
#List to save the orginal number for each area when we consider all forests
list_to_save = list(range(0,len(df_only_geo)))

for cluster in cluster_ids:
    # Get the wind value at the mean point of the cluster
    x = int((mean_coord[cluster][0] - presence_forest1_gt[0])/presence_forest1_gt[1])
    #print(x)
    y = int((mean_coord[cluster][1] - presence_forest1_gt[3])/presence_forest1_gt[5])
    #print(y)

    x_start = x - pixels
    y_start = y - pixels
    
    k = 0
    kk = 0
    i=0
    j=0
    while i < len(pixels_list):
        x_start = x_start + 1
        #print ('x', x_start)
        j=0
        while j < len(pixels_list):
            presence_forest12[k] = presence_forest1_data[y_start+j, x_start]        
            #print(presence_forest2[k])
            #print(y_start+j, x_start)  
            #print(k, j)
            k = k+1
            j = j+1
            #print('termine', presence_forest2[k])
        
        i = i+1   

    list_to_save[cluster] = sum(presence_forest12)
    presence_forest1[cluster] = sum(presence_forest12)/value_total_pixels #This gives % of forests, 
    #the lower the number, the less forest thre is
df_only_geo.insert(14, 'Presence of forests (all)', presence_forest1)
df_only_geo

,index,Interview ID,Village,lat,long,"long,lat",age,sex,no_people_in_household,highest_grade_completed,...,Closest Rivers (km),Closest Nat/Subnat Road (km),Closest National Road (km),Land Cover,Luminance,Luminance Band 3,Luminance Band 2,Luminance Band 1,Travel Time (min),Precipitation (mm)
0,1,60645a36a2237b3f54690be5,Mabwe,0.595131,30.153128,"(30.15312827, 0.5951310726)",35.0,2.0,3.0,2.0,...,0.801532,0.621699,4.495595,40.0,17.4412,29.0,17.0,15.0,24.0,1482.106567
1,8,60645af8a2237b3f5469146c,Bihondo,0.578531,30.170566,"(30.1705658, 0.5785313656)",45.0,1.0,7.0,0.0,...,0.825146,1.330831,1.842590,40.0,17.2286,29.0,17.0,14.0,19.0,1482.106567
2,9,60645921a2237b3f5468ffc1,Okabi,3.125250,30.848807,"(30.84880735, 3.125250327)",64.0,2.0,4.0,0.0,...,1.018604,5.073637,5.073637,40.0,18.5856,32.0,18.0,16.0,21.0,1345.285400
3,15,60d06f3edefc3909beb58988,Ngoro,0.196600,30.168300,"(30.1683, 0.1966)",28.0,1.0,4.0,4.0,...,0.630554,6.338110,6.338110,126.0,15.2286,27.0,15.0,12.0,28.0,891.146545
4,22,60d06f50defc3909beb589a0,Kyigabo,0.264200,30.108400,"(30.1084, 0.2642)",35.0,1.0,5.0,2.0,...,0.221976,0.355968,1.180557,40.0,17.6578,32.0,17.0,15.0,13.0,977.963806
5,25,60d06f53defc3909beb589a9,Kazinga,0.264200,30.108400,"(30.1084, 0.2642)",31.0,2.0,5.0,0.0,...,0.221976,0.355968,1.180557,40.0,17.6578,32.0,17.0,15.0,13.0,977.963806
6,26,60d06f60defc3909beb589b1,Kitumba,0.589700,30.254900,"(30.2549, 0.5897)",37.0,1.0,4.0,1.0,...,0.422972,3.752815,3.752815,20.0,15.8716,26.0,16.0,12.0,10.0,1247.092651
7,30,60d06f63defc3909beb589bc,Kakanju,0.542400,30.196500,"(30.1965, 0.5424)",28.0,2.0,3.0,2.0,...,0.036051,2.745248,2.745248,30.0,16.1564,27.0,16.0,13.0,21.0,1482.106567
8,31,60d06f64defc3909beb589be,Kaburegye,0.542400,30.196500,"(30.1965, 0.5424)",45.0,2.0,4.0,0.0,...,0.036051,2.745248,2.745248,30.0,16.1564,27.0,16.0,13.0,21.0,1482.106567
9,35,60d06f67defc3909beb589c6,Karambi,0.169900,30.078100,"(30.0781, 0.1699)",26.0,2.0,3.0,2.0,...,1.375503,0.530190,0.530190,50.0,22.2314,41.0,21.0,20.0,0.0,1003.117432


In [64]:
#Get the final table 
df_result = pd.concat([df_only_geo, df_new_data_order], axis=1)
df_result.to_excel("00._output_v3_Observations_falses.xlsx")
df_result.head(1)

,index,Interview ID,Village,lat,long,"long,lat",age,sex,no_people_in_household,highest_grade_completed,...,Unburden,Comfort,Water Security,Medical treatment,Appeal (senses),Personal performance,Aspiration,Time management,Income,Reliability
0,1,60645a36a2237b3f54690be5,Mabwe,0.595131,30.153128,"(30.15312827, 0.5951310726)",35.0,2.0,3.0,2.0,...,1,1,0,0,1,0,1,0,1,1


# Only presence of forest (reserves ans protected areas not included)

In [65]:
# Filepaths to raster
#presence_forest_raster = "only_forest_uganda.tif"
presence_forest2_raster = "forest_without_reserves.tif"
#https://lcviewer.vito.be/2016/Uganda
#https://lcviewer.vito.be/download   (0 to 100, with 0 'no forest')

In [66]:
presence_forest2 = list(range(0,len(df_only_geo)))

walk = 10000 #Distance in meters that a person walks to get firewood
distance_x_pixel = 100 #meters(measured in QGIS = 110.383)
distance_y_pixel = 100 #meters (measured in QGIS = 109.841)
distance_pixel = (distance_x_pixel + distance_y_pixel) / 2
pixels = round(walk / distance_pixel)
presence_forest22 = list(range(0,pixels*2*pixels*2))
total_pixels = pixels*2*pixels*2
value_total_pixels = 100 * total_pixels

#print(len(presence_forest2))
#print('Distance pixel prom:', distance_pixel, 'Pixels:', pixels)

# Read in travel time data
presence_forest2_gdata = gdal.Open(presence_forest2_raster)
presence_forest2_gt = presence_forest2_gdata.GetGeoTransform()
presence_forest2_data = presence_forest2_gdata.ReadAsArray().astype(np.float)
#display(presence_forest_gt)

# Fake list of cluster_ids
cluster_ids = list(range(0,len(df_only_geo)))

#Lists for getting the data of around 10 km of the point
pixels_list = list(range(0,pixels*2))


# Fake list of mean coordinates
mean_coord = list(df_only_geo['long,lat'])
#print(type(mean_coord[0]))

# Get information about each cluster. For reference: in the geotransform loaded from GDAL,
# [0] = top left x, [1] = w-e pixel resolution, [2] = 0, [3] = top left y, [4] = 0, [5] = n-s pixel resolution

#(28.76875, 0.00025000000000000006, 0.0, 4.9975, 0.0, -0.00025)

In [67]:
for cluster in cluster_ids:
    # Get the wind value at the mean point of the cluster
    x = int((mean_coord[cluster][0] - presence_forest2_gt[0])/presence_forest2_gt[1])
    #print(x)
    y = int((mean_coord[cluster][1] - presence_forest2_gt[3])/presence_forest2_gt[5])
    #print(y)

    x_start = x - pixels
    y_start = y - pixels
    
    k = 0
    kk = 0
    i=0
    j=0
    while i < len(pixels_list):
        x_start = x_start + 1
        #print ('x', x_start)
        j=0
        while j < len(pixels_list):
            presence_forest22[k] = presence_forest2_data[y_start+j, x_start]        
            #print(presence_forest2[k])
            #print(y_start+j, x_start)  
            #print(k, j)
            k = k+1
            j = j+1
            #print('termine', presence_forest2[k])
        
        i = i+1   
    presence_forest2[cluster] = sum(presence_forest22)/(value_total_pixels-(list_to_save[cluster]-sum(presence_forest22)))
    #print('this', sum(presence_forest2))
    #print('this', statistics.mean(presence_forest2))
    #display(presence_forest2)                                          

df_only_geo.insert(14, 'Presence of forests (no reserves)', presence_forest2)
df_only_geo

,index,Interview ID,Village,lat,long,"long,lat",age,sex,no_people_in_household,highest_grade_completed,...,Closest Rivers (km),Closest Nat/Subnat Road (km),Closest National Road (km),Land Cover,Luminance,Luminance Band 3,Luminance Band 2,Luminance Band 1,Travel Time (min),Precipitation (mm)
0,1,60645a36a2237b3f54690be5,Mabwe,0.595131,30.153128,"(30.15312827, 0.5951310726)",35.0,2.0,3.0,2.0,...,0.801532,0.621699,4.495595,40.0,17.4412,29.0,17.0,15.0,24.0,1482.106567
1,8,60645af8a2237b3f5469146c,Bihondo,0.578531,30.170566,"(30.1705658, 0.5785313656)",45.0,1.0,7.0,0.0,...,0.825146,1.330831,1.842590,40.0,17.2286,29.0,17.0,14.0,19.0,1482.106567
2,9,60645921a2237b3f5468ffc1,Okabi,3.125250,30.848807,"(30.84880735, 3.125250327)",64.0,2.0,4.0,0.0,...,1.018604,5.073637,5.073637,40.0,18.5856,32.0,18.0,16.0,21.0,1345.285400
3,15,60d06f3edefc3909beb58988,Ngoro,0.196600,30.168300,"(30.1683, 0.1966)",28.0,1.0,4.0,4.0,...,0.630554,6.338110,6.338110,126.0,15.2286,27.0,15.0,12.0,28.0,891.146545
4,22,60d06f50defc3909beb589a0,Kyigabo,0.264200,30.108400,"(30.1084, 0.2642)",35.0,1.0,5.0,2.0,...,0.221976,0.355968,1.180557,40.0,17.6578,32.0,17.0,15.0,13.0,977.963806
5,25,60d06f53defc3909beb589a9,Kazinga,0.264200,30.108400,"(30.1084, 0.2642)",31.0,2.0,5.0,0.0,...,0.221976,0.355968,1.180557,40.0,17.6578,32.0,17.0,15.0,13.0,977.963806
6,26,60d06f60defc3909beb589b1,Kitumba,0.589700,30.254900,"(30.2549, 0.5897)",37.0,1.0,4.0,1.0,...,0.422972,3.752815,3.752815,20.0,15.8716,26.0,16.0,12.0,10.0,1247.092651
7,30,60d06f63defc3909beb589bc,Kakanju,0.542400,30.196500,"(30.1965, 0.5424)",28.0,2.0,3.0,2.0,...,0.036051,2.745248,2.745248,30.0,16.1564,27.0,16.0,13.0,21.0,1482.106567
8,31,60d06f64defc3909beb589be,Kaburegye,0.542400,30.196500,"(30.1965, 0.5424)",45.0,2.0,4.0,0.0,...,0.036051,2.745248,2.745248,30.0,16.1564,27.0,16.0,13.0,21.0,1482.106567
9,35,60d06f67defc3909beb589c6,Karambi,0.169900,30.078100,"(30.0781, 0.1699)",26.0,2.0,3.0,2.0,...,1.375503,0.530190,0.530190,50.0,22.2314,41.0,21.0,20.0,0.0,1003.117432


In [68]:
#Get the final table 
df_result = pd.concat([df_only_geo, df_new_data_order], axis=1)
df_result.to_excel("00._output_v3_Observations_falses.xlsx")
df_result.head(1)

,index,Interview ID,Village,lat,long,"long,lat",age,sex,no_people_in_household,highest_grade_completed,...,Unburden,Comfort,Water Security,Medical treatment,Appeal (senses),Personal performance,Aspiration,Time management,Income,Reliability
0,1,60645a36a2237b3f54690be5,Mabwe,0.595131,30.153128,"(30.15312827, 0.5951310726)",35.0,2.0,3.0,2.0,...,1,1,0,0,1,0,1,0,1,1


# Fuel Stations Proximity

In [69]:
#https://energydata.info/dataset/national-boundary-2014
#https://overpass-turbo.eu/#
#amenity=fuel

In [70]:
#Read substations geographic data and extract relevant parameters
station_geo = "fuel_stations_Uganda.geojson"
df_station_predata_geo = gpd.read_file(station_geo)
df_station_data_geo = df_station_predata_geo[['geometry']].copy()
df_station_data_geo

,geometry
0,POINT (29.99362 -1.25951)
1,POINT (33.18130 0.44070)
2,POINT (33.02654 0.58223)
3,POINT (32.52532 0.30296)
4,POINT (32.54831 0.33521)
...,...
1062,POINT (30.13848 -0.55094)
1063,POINT (29.92646 -0.78706)
1064,POINT (32.62598 0.38553)
1065,POINT (30.95488 3.39576)


In [71]:
#Conversion of 'geometry' column in distribution_substations_2016.geojson into a dataframe
list_of_tuples=list(range(0,len(df_station_data_geo)))
i=0
while i < len(df_station_data_geo):
    point = df_station_data_geo.loc[df_station_data_geo.index.values[i], 'geometry'].coords
    list_of_tuples[i]=list(point)
    i=i+1

points = gpd.GeoDataFrame(list_of_tuples, columns=["x"])
points2 = points['x'].tolist()
points_station = gpd.GeoDataFrame(points2, columns=["Longitude","Latitude"])
#points_station

In [72]:
#List of projected coordinates for communities in Stephi's data set (df_only_geo)

lat_comm = list(df_only_geo['lat'])
long_comm = list(df_only_geo['long'])

long_comm_c = list(range(0,len(df_only_geo)))
lat_comm_c = list(range(0,len(df_only_geo)))

i=0

proj_wgs84 = pyproj.Proj(init="epsg:4326")
proj_gk4 = pyproj.Proj(init="epsg:3857")

for (long_comm,lat_comm) in zip(long_comm,lat_comm):
    long_comm_c[i], lat_comm_c[i] = pyproj.transform(proj_wgs84, proj_gk4, long_comm, lat_comm)
    i=i+1
    
#display(long_comm_c)
#display(lat_comm_c)

#List of projected coordinates from fuel stations in (df_station_data_geo)

lat_station = list(points_station['Latitude'])
long_station = list(points_station['Longitude'])

long_station_c = list(range(0,len(points_station)))
lat_station_c = list(range(0,len(points_station)))

i=0

proj_wgs84 = pyproj.Proj(init="epsg:4326")
proj_gk4 = pyproj.Proj(init="epsg:3857")

for (long_station,lat_station) in zip(long_station,lat_station):
    long_station_c[i], lat_station_c[i] = pyproj.transform(proj_wgs84, proj_gk4, long_station, lat_station)
    i=i+1
    
#display(long_station_c)
#display(lat_station_c)

In [73]:
## Get the distance from each community point to the nearest fuel station

d = {'long_c (comm)':long_comm_c,'lat_c (comm)':lat_comm_c}
communities = gpd.GeoDataFrame(d)

d2 = {'long_c (sta)':long_station_c,'lat_c (sta)':lat_station_c}
stations = gpd.GeoDataFrame(d2)

# Create a BallTree 
tree = BallTree(stations[['long_c (sta)', 'lat_c (sta)']].values, leaf_size=2)

# Query the BallTree on each feature from 'communities' to find the distance
# to the nearest 'station' and its id
communities['distance_nearest'], communities['id_nearest'] = tree.query(
    communities[['long_c (comm)', 'lat_c (comm)']].values, # The input array for the query
    k=1, # The number of nearest neighbors
)

k=0

while k < len(communities):
    communities.loc[communities.index.values[k], 'distance_nearest'] =  communities.loc[communities.index.values[k], 'distance_nearest']/1000
    k=k+1
#display(communities)

distance_fuel_station = communities['distance_nearest'].tolist()
   
df_only_geo.insert(14, 'Closest Fuel Station (km)', distance_fuel_station)
df_only_geo

,index,Interview ID,Village,lat,long,"long,lat",age,sex,no_people_in_household,highest_grade_completed,...,Closest Rivers (km),Closest Nat/Subnat Road (km),Closest National Road (km),Land Cover,Luminance,Luminance Band 3,Luminance Band 2,Luminance Band 1,Travel Time (min),Precipitation (mm)
0,1,60645a36a2237b3f54690be5,Mabwe,0.595131,30.153128,"(30.15312827, 0.5951310726)",35.0,2.0,3.0,2.0,...,0.801532,0.621699,4.495595,40.0,17.4412,29.0,17.0,15.0,24.0,1482.106567
1,8,60645af8a2237b3f5469146c,Bihondo,0.578531,30.170566,"(30.1705658, 0.5785313656)",45.0,1.0,7.0,0.0,...,0.825146,1.330831,1.842590,40.0,17.2286,29.0,17.0,14.0,19.0,1482.106567
2,9,60645921a2237b3f5468ffc1,Okabi,3.125250,30.848807,"(30.84880735, 3.125250327)",64.0,2.0,4.0,0.0,...,1.018604,5.073637,5.073637,40.0,18.5856,32.0,18.0,16.0,21.0,1345.285400
3,15,60d06f3edefc3909beb58988,Ngoro,0.196600,30.168300,"(30.1683, 0.1966)",28.0,1.0,4.0,4.0,...,0.630554,6.338110,6.338110,126.0,15.2286,27.0,15.0,12.0,28.0,891.146545
4,22,60d06f50defc3909beb589a0,Kyigabo,0.264200,30.108400,"(30.1084, 0.2642)",35.0,1.0,5.0,2.0,...,0.221976,0.355968,1.180557,40.0,17.6578,32.0,17.0,15.0,13.0,977.963806
5,25,60d06f53defc3909beb589a9,Kazinga,0.264200,30.108400,"(30.1084, 0.2642)",31.0,2.0,5.0,0.0,...,0.221976,0.355968,1.180557,40.0,17.6578,32.0,17.0,15.0,13.0,977.963806
6,26,60d06f60defc3909beb589b1,Kitumba,0.589700,30.254900,"(30.2549, 0.5897)",37.0,1.0,4.0,1.0,...,0.422972,3.752815,3.752815,20.0,15.8716,26.0,16.0,12.0,10.0,1247.092651
7,30,60d06f63defc3909beb589bc,Kakanju,0.542400,30.196500,"(30.1965, 0.5424)",28.0,2.0,3.0,2.0,...,0.036051,2.745248,2.745248,30.0,16.1564,27.0,16.0,13.0,21.0,1482.106567
8,31,60d06f64defc3909beb589be,Kaburegye,0.542400,30.196500,"(30.1965, 0.5424)",45.0,2.0,4.0,0.0,...,0.036051,2.745248,2.745248,30.0,16.1564,27.0,16.0,13.0,21.0,1482.106567
9,35,60d06f67defc3909beb589c6,Karambi,0.169900,30.078100,"(30.0781, 0.1699)",26.0,2.0,3.0,2.0,...,1.375503,0.530190,0.530190,50.0,22.2314,41.0,21.0,20.0,0.0,1003.117432


In [74]:
#Get the final table 
df_result = pd.concat([df_only_geo, df_new_data_order], axis=1)
df_result.to_excel("00._output_v3_Observations_falses.xlsx")
df_result.head(1)

,index,Interview ID,Village,lat,long,"long,lat",age,sex,no_people_in_household,highest_grade_completed,...,Unburden,Comfort,Water Security,Medical treatment,Appeal (senses),Personal performance,Aspiration,Time management,Income,Reliability
0,1,60645a36a2237b3f54690be5,Mabwe,0.595131,30.153128,"(30.15312827, 0.5951310726)",35.0,2.0,3.0,2.0,...,1,1,0,0,1,0,1,0,1,1


# Proximity to Lakes

In [75]:
#Get coordinates from the polygons
import pyproj
from shapely.ops import transform

project = pyproj.Transformer.from_proj(
    pyproj.Proj(init='epsg:4326'), # source coordinate system
    pyproj.Proj(init='epsg:3857')) # destination coordinate system

# g1 is a shapley Polygon
lake_geo = "Uganda_Lakes.shp"
df_lake_predata_geo = gpd.read_file(lake_geo)
g1 = df_lake_predata_geo[['geometry']].copy()
#display(g1)

lista_poly= list(range(0,len(g1)))
  
for i in lista_poly:
    lista_poly[i] = transform(project.transform, g1.loc[g1.index.values[i], 'geometry'])  # apply projection

geometry2 = gpd.GeoDataFrame(lista_poly, columns=["geometry"])
geometry2

,geometry
0,"POLYGON ((3781715.297 177409.301, 3781714.812 ..."
1,"POLYGON ((3803880.180 185763.032, 3803840.834 ..."
2,"POLYGON ((3715168.740 147766.711, 3715168.248 ..."
3,"POLYGON ((3722821.438 152959.434, 3722820.944 ..."
4,"POLYGON ((3731177.926 146865.408, 3731177.660 ..."
...,...
98,"POLYGON ((3551174.132 -37724.165, 3551329.826 ..."
99,"POLYGON ((3695324.469 46250.531, 3695323.984 4..."
100,"POLYGON ((3595834.250 180234.156, 3595833.781 ..."
101,"POLYGON ((3499694.562 258970.937, 3499686.125 ..."


In [76]:
#List of projected coordinates for communities in Stephi's data set (df_only_geo)

lat_comm = list(df_only_geo['lat'])
long_comm = list(df_only_geo['long'])

#print(type(lat_comm[3])) 

long_comm_c = list(range(0,len(df_only_geo)))
lat_comm_c = list(range(0,len(df_only_geo)))

i=0

proj_wgs84 = pyproj.Proj(init="epsg:4326")
proj_gk4 = pyproj.Proj(init="epsg:3857")

for (long_comm,lat_comm) in zip(long_comm,lat_comm):
    long_comm_c[i], lat_comm_c[i] = pyproj.transform(proj_wgs84, proj_gk4, long_comm, lat_comm)
    i=i+1
    
#display(long_comm_c)
#display(lat_comm_c)

#List of projected coordinates from substations in Substation data (df_sub_data_geo)

#lat_sub = list(df_sub_data_geo['Latitude'])
#long_sub = list(df_sub_data_geo['Longitude'])

#long_sub_c = list(range(0,len(df_sub_data_geo)))
#lat_sub_c = list(range(0,len(df_sub_data_geo)))

#i=0

#proj_wgs84 = pyproj.Proj(init="epsg:4326")
#proj_gk4 = pyproj.Proj(init="epsg:3857")

#for (long_sub,lat_sub) in zip(long_sub,lat_sub):
    #long_sub_c[i], lat_sub_c[i] = pyproj.transform(proj_wgs84, proj_gk4, long_sub, lat_sub)
    #i=i+1
    
#display(long_sub_c)
#display(lat_sub_c)
#print(type(lat_comm_c[3])) 
#lat_comm

In [77]:
# Instantiate list to hold travel time info
closest_lake = list(range(0,len(df_only_geo)))

cluster_ids = list(range(0,len(df_only_geo)))
radius = 25000000 # metres

for cluster in cluster_ids:
    point = Point(long_comm_c[cluster], lat_comm_c[cluster])
    buffer = point.buffer(radius)
    clipped_lakes = gpd.clip(geometry2, buffer)
    
    # calculate distance for all lines
    distances = clipped_lakes["geometry"].map(lambda line: point.distance(line))
    clipped_lakes = clipped_lakes.assign(Distance=distances)

    # get 4 roads closest to the point
    clipped_lakes = clipped_lakes.sort_values(by="Distance")
    closest_lakes = clipped_lakes.head(4)  
    
    closest_lake[cluster]=closest_lakes.loc[closest_lakes.index.values[0], 'Distance']/1000
    #display(closest_lake)
     
df_only_geo.insert(14, 'Closest lake (km)', closest_lake)
df_only_geo

,index,Interview ID,Village,lat,long,"long,lat",age,sex,no_people_in_household,highest_grade_completed,...,Closest Rivers (km),Closest Nat/Subnat Road (km),Closest National Road (km),Land Cover,Luminance,Luminance Band 3,Luminance Band 2,Luminance Band 1,Travel Time (min),Precipitation (mm)
0,1,60645a36a2237b3f54690be5,Mabwe,0.595131,30.153128,"(30.15312827, 0.5951310726)",35.0,2.0,3.0,2.0,...,0.801532,0.621699,4.495595,40.0,17.4412,29.0,17.0,15.0,24.0,1482.106567
1,8,60645af8a2237b3f5469146c,Bihondo,0.578531,30.170566,"(30.1705658, 0.5785313656)",45.0,1.0,7.0,0.0,...,0.825146,1.330831,1.842590,40.0,17.2286,29.0,17.0,14.0,19.0,1482.106567
2,9,60645921a2237b3f5468ffc1,Okabi,3.125250,30.848807,"(30.84880735, 3.125250327)",64.0,2.0,4.0,0.0,...,1.018604,5.073637,5.073637,40.0,18.5856,32.0,18.0,16.0,21.0,1345.285400
3,15,60d06f3edefc3909beb58988,Ngoro,0.196600,30.168300,"(30.1683, 0.1966)",28.0,1.0,4.0,4.0,...,0.630554,6.338110,6.338110,126.0,15.2286,27.0,15.0,12.0,28.0,891.146545
4,22,60d06f50defc3909beb589a0,Kyigabo,0.264200,30.108400,"(30.1084, 0.2642)",35.0,1.0,5.0,2.0,...,0.221976,0.355968,1.180557,40.0,17.6578,32.0,17.0,15.0,13.0,977.963806
5,25,60d06f53defc3909beb589a9,Kazinga,0.264200,30.108400,"(30.1084, 0.2642)",31.0,2.0,5.0,0.0,...,0.221976,0.355968,1.180557,40.0,17.6578,32.0,17.0,15.0,13.0,977.963806
6,26,60d06f60defc3909beb589b1,Kitumba,0.589700,30.254900,"(30.2549, 0.5897)",37.0,1.0,4.0,1.0,...,0.422972,3.752815,3.752815,20.0,15.8716,26.0,16.0,12.0,10.0,1247.092651
7,30,60d06f63defc3909beb589bc,Kakanju,0.542400,30.196500,"(30.1965, 0.5424)",28.0,2.0,3.0,2.0,...,0.036051,2.745248,2.745248,30.0,16.1564,27.0,16.0,13.0,21.0,1482.106567
8,31,60d06f64defc3909beb589be,Kaburegye,0.542400,30.196500,"(30.1965, 0.5424)",45.0,2.0,4.0,0.0,...,0.036051,2.745248,2.745248,30.0,16.1564,27.0,16.0,13.0,21.0,1482.106567
9,35,60d06f67defc3909beb589c6,Karambi,0.169900,30.078100,"(30.0781, 0.1699)",26.0,2.0,3.0,2.0,...,1.375503,0.530190,0.530190,50.0,22.2314,41.0,21.0,20.0,0.0,1003.117432


In [78]:
#Get the final table 
df_result = pd.concat([df_only_geo, df_new_data_order], axis=1)
df_result.to_excel("00._output_v3_Observations_falses.xlsx")
df_result.head(1)

,index,Interview ID,Village,lat,long,"long,lat",age,sex,no_people_in_household,highest_grade_completed,...,Unburden,Comfort,Water Security,Medical treatment,Appeal (senses),Personal performance,Aspiration,Time management,Income,Reliability
0,1,60645a36a2237b3f54690be5,Mabwe,0.595131,30.153128,"(30.15312827, 0.5951310726)",35.0,2.0,3.0,2.0,...,1,1,0,0,1,0,1,0,1,1


# DSH Clusters Proximity Identification

In [79]:
## Get the distance from each community point to the nearest cluster point from DHS data 

d = {'long_c (comm)':long_comm_c,'lat_c (comm)':lat_comm_c}
communities = gpd.GeoDataFrame(d)

d2 = {'long_c (clus)':long_clus_c,'lat_c (clus)':lat_clus_c}
clusters = gpd.GeoDataFrame(d2)

# Create a BallTree 
tree = BallTree(clusters[['long_c (clus)', 'lat_c (clus)']].values, leaf_size=2)

# Query the BallTree on each feature from 'communities' to find the distance
# to the nearest 'cluster' and its id
communities['distance_nearest'], communities['id_nearest'] = tree.query(
    communities[['long_c (comm)', 'lat_c (comm)']].values, # The input array for the query
    k=1, # The number of nearest neighbors
)
#display(clusters)
#display(communities)


#Look for the DHS cluster for each observation
i=0
while i < len(communities):
    #print('----------soyi', i)
    id_nearest = communities.loc[communities.index.values[i], 'id_nearest']
    #print(id_nearest)
    j=0
    while j < len(df_dhs_data_geo):
        #print('-soyj', j)
        if id_nearest == j:
            communities.loc[communities.index.values[i], 'Cluster id'] = df_dhs_data_geo.loc[df_dhs_data_geo.index.values[j], 'DHSCLUST']
            break
        else:
            j=j+1        
    i=i+1       
communities

,long_c (comm),lat_c (comm),distance_nearest,id_nearest,Cluster id
0,3.356631e+06,66250.879266,4195.886348,530,532.0
1,3.358572e+06,64402.911402,4453.903430,530,532.0
2,3.434074e+06,348073.919332,1601.005473,438,440.0
3,3.358320e+06,21885.454836,7614.573367,559,561.0
4,3.351652e+06,29410.713693,6097.303479,540,542.0
5,3.351652e+06,29410.713693,6097.303479,540,542.0
6,3.367960e+06,65646.262711,3331.071621,555,557.0
7,3.361459e+06,60380.593675,1937.407062,524,526.0
8,3.361459e+06,60380.593675,1937.407062,524,526.0
9,3.348279e+06,18913.209203,1528.656826,558,560.0


In [80]:
#Inclusion of DHS cluster number in the observations
DHS_clusters = communities['Cluster id'].tolist()
df_only_geo.insert(31, 'DHS Cluster ', DHS_clusters)
df_only_geo

,index,Interview ID,Village,lat,long,"long,lat",age,sex,no_people_in_household,highest_grade_completed,...,Closest Nat/Subnat Road (km),Closest National Road (km),Land Cover,Luminance,Luminance Band 3,Luminance Band 2,Luminance Band 1,DHS Cluster,Travel Time (min),Precipitation (mm)
0,1,60645a36a2237b3f54690be5,Mabwe,0.595131,30.153128,"(30.15312827, 0.5951310726)",35.0,2.0,3.0,2.0,...,0.621699,4.495595,40.0,17.4412,29.0,17.0,15.0,532.0,24.0,1482.106567
1,8,60645af8a2237b3f5469146c,Bihondo,0.578531,30.170566,"(30.1705658, 0.5785313656)",45.0,1.0,7.0,0.0,...,1.330831,1.842590,40.0,17.2286,29.0,17.0,14.0,532.0,19.0,1482.106567
2,9,60645921a2237b3f5468ffc1,Okabi,3.125250,30.848807,"(30.84880735, 3.125250327)",64.0,2.0,4.0,0.0,...,5.073637,5.073637,40.0,18.5856,32.0,18.0,16.0,440.0,21.0,1345.285400
3,15,60d06f3edefc3909beb58988,Ngoro,0.196600,30.168300,"(30.1683, 0.1966)",28.0,1.0,4.0,4.0,...,6.338110,6.338110,126.0,15.2286,27.0,15.0,12.0,561.0,28.0,891.146545
4,22,60d06f50defc3909beb589a0,Kyigabo,0.264200,30.108400,"(30.1084, 0.2642)",35.0,1.0,5.0,2.0,...,0.355968,1.180557,40.0,17.6578,32.0,17.0,15.0,542.0,13.0,977.963806
5,25,60d06f53defc3909beb589a9,Kazinga,0.264200,30.108400,"(30.1084, 0.2642)",31.0,2.0,5.0,0.0,...,0.355968,1.180557,40.0,17.6578,32.0,17.0,15.0,542.0,13.0,977.963806
6,26,60d06f60defc3909beb589b1,Kitumba,0.589700,30.254900,"(30.2549, 0.5897)",37.0,1.0,4.0,1.0,...,3.752815,3.752815,20.0,15.8716,26.0,16.0,12.0,557.0,10.0,1247.092651
7,30,60d06f63defc3909beb589bc,Kakanju,0.542400,30.196500,"(30.1965, 0.5424)",28.0,2.0,3.0,2.0,...,2.745248,2.745248,30.0,16.1564,27.0,16.0,13.0,526.0,21.0,1482.106567
8,31,60d06f64defc3909beb589be,Kaburegye,0.542400,30.196500,"(30.1965, 0.5424)",45.0,2.0,4.0,0.0,...,2.745248,2.745248,30.0,16.1564,27.0,16.0,13.0,526.0,21.0,1482.106567
9,35,60d06f67defc3909beb589c6,Karambi,0.169900,30.078100,"(30.0781, 0.1699)",26.0,2.0,3.0,2.0,...,0.530190,0.530190,50.0,22.2314,41.0,21.0,20.0,560.0,0.0,1003.117432


In [81]:
#Get the final table 
df_result = pd.concat([df_only_geo, df_new_data_order], axis=1)
df_result.to_excel("00._output_v3_Observations_falses.xlsx")
df_result.head(1)

,index,Interview ID,Village,lat,long,"long,lat",age,sex,no_people_in_household,highest_grade_completed,...,Unburden,Comfort,Water Security,Medical treatment,Appeal (senses),Personal performance,Aspiration,Time management,Income,Reliability
0,1,60645a36a2237b3f54690be5,Mabwe,0.595131,30.153128,"(30.15312827, 0.5951310726)",35.0,2.0,3.0,2.0,...,1,1,0,0,1,0,1,0,1,1


# -------------------------------------------------------------------------------------------------------------

# DHS Data Import

In [82]:
DHS_survey = pd.read_excel("05._DHS_Individual_2016.xlsx")
df_dhs_predata_survey = pd.DataFrame(DHS_survey)

In [83]:
df_dhs_data_survey = df_dhs_predata_survey[['V001', 'V012','V190A', 'V113', 'V115', 'V106', 'V116', 'V119', 
                                            'V161', 'V136', 'V151', 'V152', 'V159', 'V158', 'V157', 'V171B']].copy()
df_dhs_data_survey.rename(columns={'V001':"Cluster id",
                                   'V190A': "Wealth Index", 
                                   'V012': "Age",
                                   'V113': 'Source of drinking water', 
                                   'V115': "Time to water source", 
                                   'V106': "Highest Educational Level", 
                                   'V116': 'Type of toilet facility', 
                                   'V119': "Electricity in Household", 
                                   'V161': "Type of cooking fuel", 
                                   'V136': "No. Household Members", 
                                   'V158': "Frequency of listening to radio",
                                   'V159': "Frequency of watching television",
                                   'V157': "Frequency of reading newspaper or magazine",
                                   'V171B': "Frequency of using internet last month"}, inplace = True)
#df_dhs_data_survey.to_excel("output_df_dhs_data_survey.xlsx")

## Wealth Index (WI)
###### From 0 to 5 

In [84]:
i=0
wealth_index = pd.DataFrame(index=range(0,len(communities)),columns=['Q1','Q2','Q3','Q4','Q5'])
while i < len(communities):
    #print('----------soyi', i)
    cluster = communities.loc[communities.index.values[i], 'Cluster id']
    #print(cluster)
    j=0
    k=0
    count = 0
    while j < len(df_dhs_data_survey):
        #print('-soyj', j)
        if cluster == df_dhs_data_survey.loc[df_dhs_data_survey.index.values[j], 'Cluster id']:
            count = count + 1
            #print('soy igual')
            my_list = list(range(0,count))
            #print(my_list)
            j=j+1
        else:
            j=j+1
            
    ii=0    
    while k < len(df_dhs_data_survey):
        #print('-soyk', k)
        if cluster == df_dhs_data_survey.loc[df_dhs_data_survey.index.values[k], 'Cluster id']:
            #print('soy igual2')
            my_list[ii] = df_dhs_data_survey.loc[df_dhs_data_survey.index.values[k], 'Wealth Index']
            #print(my_list)
            ii=ii+1
            k=k+1
        else:
            k=k+1        
    
    #To get the percentage of each quintile in relation to the total samples
    Q1 = my_list.count(1)/count
    Q2 = my_list.count(2)/count
    Q3 = my_list.count(3)/count
    Q4 = my_list.count(4)/count
    Q5 = my_list.count(5)/count
    
    wealth_index.loc[wealth_index.index.values[i], 'Q1'] = Q1
    wealth_index.loc[wealth_index.index.values[i], 'Q2'] = Q2
    wealth_index.loc[wealth_index.index.values[i], 'Q3'] = Q3
    wealth_index.loc[wealth_index.index.values[i], 'Q4'] = Q4
    wealth_index.loc[wealth_index.index.values[i], 'Q5'] = Q5
    
    i=i+1
    
list_Q1 = list(wealth_index['Q1'].tolist())
list_Q2 = list(wealth_index['Q2'].tolist())
list_Q3 = list(wealth_index['Q3'].tolist())
list_Q4 = list(wealth_index['Q4'].tolist())
list_Q5 = list(wealth_index['Q5'].tolist())

df_only_geo.insert(32, 'Q5', list_Q5)
df_only_geo.insert(32, 'Q4', list_Q4)
df_only_geo.insert(32, 'Q3', list_Q3)
df_only_geo.insert(32, 'Q2', list_Q2)
df_only_geo.insert(32, 'Q1', list_Q1)

display(wealth_index)
df_only_geo

,Q1,Q2,Q3,Q4,Q5
0,0.0,0.107143,0.357143,0.321429,0.214286
1,0.0,0.107143,0.357143,0.321429,0.214286
2,0.689655,0.103448,0.103448,0.103448,0.0
3,0.0,0.0,0.375,0.208333,0.416667
4,0.0,0.0,0.275862,0.206897,0.517241
5,0.0,0.0,0.275862,0.206897,0.517241
6,0.0,0.107143,0.25,0.428571,0.214286
7,0.076923,0.307692,0.153846,0.461538,0.0
8,0.076923,0.307692,0.153846,0.461538,0.0
9,0.055556,0.333333,0.277778,0.194444,0.138889


,index,Interview ID,Village,lat,long,"long,lat",age,sex,no_people_in_household,highest_grade_completed,...,Luminance Band 2,Luminance Band 1,DHS Cluster,Q1,Q2,Q3,Q4,Q5,Travel Time (min),Precipitation (mm)
0,1,60645a36a2237b3f54690be5,Mabwe,0.595131,30.153128,"(30.15312827, 0.5951310726)",35.0,2.0,3.0,2.0,...,17.0,15.0,532.0,0.000000,0.107143,0.357143,0.321429,0.214286,24.0,1482.106567
1,8,60645af8a2237b3f5469146c,Bihondo,0.578531,30.170566,"(30.1705658, 0.5785313656)",45.0,1.0,7.0,0.0,...,17.0,14.0,532.0,0.000000,0.107143,0.357143,0.321429,0.214286,19.0,1482.106567
2,9,60645921a2237b3f5468ffc1,Okabi,3.125250,30.848807,"(30.84880735, 3.125250327)",64.0,2.0,4.0,0.0,...,18.0,16.0,440.0,0.689655,0.103448,0.103448,0.103448,0.000000,21.0,1345.285400
3,15,60d06f3edefc3909beb58988,Ngoro,0.196600,30.168300,"(30.1683, 0.1966)",28.0,1.0,4.0,4.0,...,15.0,12.0,561.0,0.000000,0.000000,0.375000,0.208333,0.416667,28.0,891.146545
4,22,60d06f50defc3909beb589a0,Kyigabo,0.264200,30.108400,"(30.1084, 0.2642)",35.0,1.0,5.0,2.0,...,17.0,15.0,542.0,0.000000,0.000000,0.275862,0.206897,0.517241,13.0,977.963806
5,25,60d06f53defc3909beb589a9,Kazinga,0.264200,30.108400,"(30.1084, 0.2642)",31.0,2.0,5.0,0.0,...,17.0,15.0,542.0,0.000000,0.000000,0.275862,0.206897,0.517241,13.0,977.963806
6,26,60d06f60defc3909beb589b1,Kitumba,0.589700,30.254900,"(30.2549, 0.5897)",37.0,1.0,4.0,1.0,...,16.0,12.0,557.0,0.000000,0.107143,0.250000,0.428571,0.214286,10.0,1247.092651
7,30,60d06f63defc3909beb589bc,Kakanju,0.542400,30.196500,"(30.1965, 0.5424)",28.0,2.0,3.0,2.0,...,16.0,13.0,526.0,0.076923,0.307692,0.153846,0.461538,0.000000,21.0,1482.106567
8,31,60d06f64defc3909beb589be,Kaburegye,0.542400,30.196500,"(30.1965, 0.5424)",45.0,2.0,4.0,0.0,...,16.0,13.0,526.0,0.076923,0.307692,0.153846,0.461538,0.000000,21.0,1482.106567
9,35,60d06f67defc3909beb589c6,Karambi,0.169900,30.078100,"(30.0781, 0.1699)",26.0,2.0,3.0,2.0,...,21.0,20.0,560.0,0.055556,0.333333,0.277778,0.194444,0.138889,0.0,1003.117432


In [85]:
#Get the final table 
df_result = pd.concat([df_only_geo, df_new_data_order], axis=1)
df_result.to_excel("00._output_v3_Observations_falses.xlsx")
df_result.head(1)

,index,Interview ID,Village,lat,long,"long,lat",age,sex,no_people_in_household,highest_grade_completed,...,Unburden,Comfort,Water Security,Medical treatment,Appeal (senses),Personal performance,Aspiration,Time management,Income,Reliability
0,1,60645a36a2237b3f54690be5,Mabwe,0.595131,30.153128,"(30.15312827, 0.5951310726)",35.0,2.0,3.0,2.0,...,1,1,0,0,1,0,1,0,1,1
